# **<font color='orange'> Dataset Cardinality vs Fidelity</font>**

Here, we compare the quantum state reconstruction fidelities when datasets of different cardinalities are considered.

# **<font color='orange'> Libraries and Random Seeds</font>**

In [1]:
import numpy as np
import scipy as sp
import scipy.linalg
import matplotlib.pyplot as plt
import qiskit as qk
import qiskit.visualization

from qiskit import Aer
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import Statevector

from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit import transpile
from qiskit.quantum_info.operators import Operator

import tensorflow as tf
print("TensorFlow version:", tf.__version__)
from tensorflow.python.framework.ops import convert_to_tensor

TensorFlow version: 2.11.0


In [2]:
# Ensure reproducibility
np.random.seed(123)
tf.random.set_seed(123)
import random
random.seed(10)

# **<font color='orange'> Functions</font>**

In [3]:
def random_unitary(N):
    """
    Generation of an NxN random unitary matrix.
    """
    Z=np.random.randn(N,N) + 1.0j * np.random.randn(N,N)
    [Q,R]=sp.linalg.qr(Z)
    D=np.diag(np.diagonal(R)/np.abs(np.diagonal(R)))
    return np.dot(Q,D)

In [4]:
def get_density_matrix(state_vector):
    """
    Compute the density matrix from the state vector: 
       rho = |psi >< psi|.
    """
    density_matrix = np.outer(state_vector, np.conjugate(state_vector))
    return density_matrix

In [5]:
# Here we define the identity matrix and the Pauli matrices for dimension 2 (one qubit)

I = np.array([[1, 0],[0, 1]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

# Function which computes the components of the Bloch vector, given the density matrix 

def Bloch_vector(rho):
    """
    Compute the components of the Bloch vector, given the density matrix rho:
      r_i = Tr[rho * i].
    """
    ax = np.trace(np.dot(rho, X)).real
    ay = np.trace(np.dot(rho, Y)).real
    az = np.trace(np.dot(rho, Z)).real
    pnt = [ax, ay, az]
    return np.array(pnt)

In [6]:
# Define the simulator
sim_bknd=Aer.get_backend('aer_simulator')

In [7]:
def generate_Haar_data(num_qubits, samples=1000):
    """
    Generate "samples" Haar distributed data, for "num_qubits"- qubit states.
    """
    data = []
    for i in range(samples):
        qc = qk.QuantumCircuit(num_qubits) # create a quantum circuit with "num_qubits" qubits
        u = random_unitary(2**num_qubits)
        qc.unitary(u, qubits=range(num_qubits)) # apply the random unitary transformation to the circuit
        qc = qk.transpile(qc, backend=sim_bknd) # optimize the circuit
        qc.save_statevector() # save the state vector obtained by the simulation

        state = sim_bknd.run(qc).result().get_statevector(qc) # simulation and get the state vector
        state = np.asarray(state)
        data.append(state) 
    return data

In [8]:
def density_matrix_from_bloch_vector(bloch_vector):
    """
    Compute the density matrix given the Bloch vector:
    rho = 0.5 * (I + r_x * X + r_y * Y + r_z * Z).
    """
    rho = 0.5 * (I+ bloch_vector[0]*X + bloch_vector[1]*Y + bloch_vector[2]*Z)
    return rho

In [9]:
# Define the Pauli basis for single qubit states
I_tf = tf.constant([1.0, 0.0,0.0,1.0],shape=(2,2), dtype = tf.complex64)
X_tf = tf.constant([0.0, 1.0, 1.0, 0.0],shape=(2,2), dtype = tf.complex64)
Y_tf = tf.constant([0.0+0j, 0.0-1j ,0.0+1j,0.0+0j],shape=(2,2), dtype = tf.complex64)
Z_tf = tf.constant([1.0, 0.0,0.0,-1.0],shape=(2,2), dtype = tf.complex64)

A = tf.stack([X_tf,Y_tf,Z_tf]) 

In [14]:
# Defining the fidelity for pure states

def fidelity_function(a,b):
    fid = tf.linalg.trace(a @ b)
    return fid

In [10]:
# Our loss function
@tf.function
def infidelity1(a,b):
   a = tf.cast(a, tf.complex64)
   b = tf.cast(b, tf.complex64)
   el_a = tf.einsum('ijk,mi->mjk',A,a) 
   el_b = tf.einsum('ijk,mi->mjk',A,b) 
   rho_a = 0.5 *(el_a + I_tf)
   rho_b = 0.5 * (el_b +I_tf)
   fidelity = tf.linalg.trace(rho_a @ rho_b) 
   infidelity = 1 - fidelity
   return infidelity 

# **<font color='orange'> Noise Models</font>**

In [11]:
# Single element of the operator-sum representation

def sum_element(rho,operator):
    element = np.dot(np.dot(operator,rho),operator.conj().T)
    return element

In [12]:
# Bit, phase flip and bit-phase flip channels

def flip_error(num_qubits, rho, error_type, p):

    E_0 = np.sqrt(1-p)*I
    if error_type == 'bit':
        E_1 = np.sqrt(p)*X
    if error_type == 'phase':
        E_1 = np.sqrt(p)*Z
    if error_type == 'bp':
        E_1 = np.sqrt(p)*Y
        
    rho_with_flip_error = sum_element(rho, E_0)+sum_element(rho, E_1)
    
    return rho_with_flip_error

# **<font color='orange'> Training with MSE</font>**

## <font color='gold'>5 samples</font>

In [16]:
predictions_5 = [] # Collect the predictions in a list

for i in range(10): # for-loop for statistics (average on 10 runs)
    data = generate_Haar_data(1, 100) 
    density_matrix_noise_free = [*map(get_density_matrix, data)] 
    bloch_vectors_noise_free = [*map(Bloch_vector, density_matrix_noise_free)]

    density_matrix_with_noise = []
    for i in range(len(data)):
        single_data_with_noise = flip_error(1, density_matrix_noise_free[i], 'phase', 0.2)
        density_matrix_with_noise.append(single_data_with_noise)
    bloch_vectors_with_noise = [*map(Bloch_vector, density_matrix_with_noise)]
    
   
    # Training, validation and test set
    x_train_list, x_val_list, x_test_list = bloch_vectors_with_noise[:5], bloch_vectors_with_noise[50:90], bloch_vectors_with_noise[90:]
    y_train_list, y_val_list, y_test_list = bloch_vectors_noise_free[:5], bloch_vectors_noise_free[50:90], bloch_vectors_noise_free[90:]


    x_train = tf.convert_to_tensor(x_train_list)
    y_train = tf.convert_to_tensor(y_train_list)


    x_val = tf.convert_to_tensor(x_val_list)
    y_val = tf.convert_to_tensor(y_val_list)

    x_test = tf.convert_to_tensor(x_test_list)
    y_test = tf.convert_to_tensor(y_test_list)
    

    model = tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(3,)),
      tf.keras.layers.Dense(64, activation='relu'), 
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(3),
      tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
       ])
    
 
    loss_fn = tf.keras.losses.mse
    adam_opt = tf.optimizers.Adam(0.001)
    model.compile(optimizer=adam_opt, 
              loss=loss_fn)
    

    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=5, epochs=200)
    score = model.evaluate(x_test,  y_test, verbose=2)
    
    y_prediction = model(x_test)
    y_prediction = tf.cast(y_prediction, tf.complex128)
    y_test = tf.cast(y_test, tf.complex128)
    
    fidelities = [] # Collecting the reconstruction fidelities
    for i in range(len(y_prediction)):
        den_mat = density_matrix_from_bloch_vector(y_prediction[i])
        den_mat = np.asarray(den_mat)


        den_mat_id = density_matrix_from_bloch_vector(y_test[i])
        den_mat_id = np.asarray(den_mat_id)
        
        fidelity = fidelity_function(den_mat_id, den_mat)
        fidelities.append(fidelity)
    fid5_c = tf.math.reduce_mean(fidelities)
    fid5 = tf.cast(fid5_c, tf.float64)
    predictions_5.append(fid5)
print(f"Fidelity between ideal and predicted Bloch vectors: {tf.math.reduce_mean(predictions_5).numpy()}")

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 0.7623 - val_loss: 0.7364
Epoch 2/200
1/1 [==============================] - 0s 58ms/step - loss: 0.6366 - val_loss: 0.7000
Epoch 3/200
1/1 [==============================] - 0s 59ms/step - loss: 0.5473 - val_loss: 0.6748
Epoch 4/200
1/1 [==============================] - 0s 63ms/step - loss: 0.4881 - val_loss: 0.6555
Epoch 5/200
1/1 [==============================] - 0s 64ms/step - loss: 0.4483 - val_loss: 0.6389
Epoch 6/200
1/1 [==============================] - 0s 56ms/step - loss: 0.4176 - val_loss: 0.6241
Epoch 7/200
1/1 [==============================] - 0s 55ms/step - loss: 0.3923 - val_loss: 0.6099
Epoch 8/200
1/1 [==============================] - 0s 60ms/step - loss: 0.3698 - val_loss: 0.5960
Epoch 9/200
1/1 [==============================] - 0s 60ms/step - loss: 0.3494 - val_loss: 0.5819
Epoch 10/200
1/1 [==============================] - 0s 53ms/step - loss: 0.3308 - val_loss: 0.5670
Epoch 11/200
1/1 [===

1/1 [==============================] - 0s 56ms/step - loss: 1.1281e-08 - val_loss: 0.1263
Epoch 162/200
1/1 [==============================] - 0s 55ms/step - loss: 1.0501e-08 - val_loss: 0.1263
Epoch 163/200
1/1 [==============================] - 0s 60ms/step - loss: 9.9039e-09 - val_loss: 0.1263
Epoch 164/200
1/1 [==============================] - 0s 53ms/step - loss: 9.6094e-09 - val_loss: 0.1263
Epoch 165/200
1/1 [==============================] - 0s 55ms/step - loss: 9.6260e-09 - val_loss: 0.1263
Epoch 166/200
1/1 [==============================] - 0s 53ms/step - loss: 9.8870e-09 - val_loss: 0.1263
Epoch 167/200
1/1 [==============================] - 0s 56ms/step - loss: 1.0253e-08 - val_loss: 0.1263
Epoch 168/200
1/1 [==============================] - 0s 53ms/step - loss: 1.0582e-08 - val_loss: 0.1263
Epoch 169/200
1/1 [==============================] - 0s 59ms/step - loss: 1.0717e-08 - val_loss: 0.1263
Epoch 170/200
1/1 [==============================] - 0s 57ms/step - loss: 1.05

Epoch 121/200
1/1 [==============================] - 0s 45ms/step - loss: 1.0938e-06 - val_loss: 0.2683
Epoch 122/200
1/1 [==============================] - 0s 47ms/step - loss: 9.4127e-07 - val_loss: 0.2683
Epoch 123/200
1/1 [==============================] - 0s 46ms/step - loss: 8.0054e-07 - val_loss: 0.2683
Epoch 124/200
1/1 [==============================] - 0s 49ms/step - loss: 6.8128e-07 - val_loss: 0.2682
Epoch 125/200
1/1 [==============================] - 0s 46ms/step - loss: 5.8600e-07 - val_loss: 0.2682
Epoch 126/200
1/1 [==============================] - 0s 44ms/step - loss: 5.1087e-07 - val_loss: 0.2682
Epoch 127/200
1/1 [==============================] - 0s 44ms/step - loss: 4.4832e-07 - val_loss: 0.2682
Epoch 128/200
1/1 [==============================] - 0s 45ms/step - loss: 3.9035e-07 - val_loss: 0.2682
Epoch 129/200
1/1 [==============================] - 0s 45ms/step - loss: 3.3134e-07 - val_loss: 0.2682
Epoch 130/200
1/1 [==============================] - 0s 45ms/ste

Epoch 200/200
1/1 [==============================] - 0s 47ms/step - loss: 1.3601e-10 - val_loss: 0.2681
1/1 - 0s - loss: 0.1835 - 112ms/epoch - 112ms/step
Epoch 1/200
1/1 [==============================] - 1s 899ms/step - loss: 1.0339 - val_loss: 0.6227
Epoch 2/200
1/1 [==============================] - 0s 49ms/step - loss: 0.8274 - val_loss: 0.5378
Epoch 3/200
1/1 [==============================] - 0s 45ms/step - loss: 0.6687 - val_loss: 0.4550
Epoch 4/200
1/1 [==============================] - 0s 47ms/step - loss: 0.5444 - val_loss: 0.3757
Epoch 5/200
1/1 [==============================] - 0s 45ms/step - loss: 0.4410 - val_loss: 0.3042
Epoch 6/200
1/1 [==============================] - 0s 49ms/step - loss: 0.3580 - val_loss: 0.2430
Epoch 7/200
1/1 [==============================] - 0s 46ms/step - loss: 0.2885 - val_loss: 0.1931
Epoch 8/200
1/1 [==============================] - 0s 45ms/step - loss: 0.2300 - val_loss: 0.1540
Epoch 9/200
1/1 [==============================] - 0s 48ms/s

1/1 [==============================] - 0s 50ms/step - loss: 1.0266e-08 - val_loss: 0.0301
Epoch 161/200
1/1 [==============================] - 0s 47ms/step - loss: 1.0585e-08 - val_loss: 0.0301
Epoch 162/200
1/1 [==============================] - 0s 47ms/step - loss: 1.1298e-08 - val_loss: 0.0301
Epoch 163/200
1/1 [==============================] - 0s 49ms/step - loss: 1.1935e-08 - val_loss: 0.0301
Epoch 164/200
1/1 [==============================] - 0s 51ms/step - loss: 1.2170e-08 - val_loss: 0.0301
Epoch 165/200
1/1 [==============================] - 0s 46ms/step - loss: 1.1830e-08 - val_loss: 0.0301
Epoch 166/200
1/1 [==============================] - 0s 43ms/step - loss: 1.0991e-08 - val_loss: 0.0301
Epoch 167/200
1/1 [==============================] - 0s 47ms/step - loss: 9.8507e-09 - val_loss: 0.0301
Epoch 168/200
1/1 [==============================] - 0s 46ms/step - loss: 8.6664e-09 - val_loss: 0.0301
Epoch 169/200
1/1 [==============================] - 0s 48ms/step - loss: 7.61

...

1/1 [==============================] - 0s 47ms/step - loss: 7.9018e-04 - val_loss: 0.2572
Epoch 120/200
1/1 [==============================] - 0s 49ms/step - loss: 7.6374e-04 - val_loss: 0.2567
Epoch 121/200
1/1 [==============================] - 0s 45ms/step - loss: 7.3630e-04 - val_loss: 0.2563
Epoch 122/200
1/1 [==============================] - 0s 48ms/step - loss: 7.1111e-04 - val_loss: 0.2558
Epoch 123/200
1/1 [==============================] - 0s 49ms/step - loss: 6.8593e-04 - val_loss: 0.2553
Epoch 124/200
1/1 [==============================] - 0s 47ms/step - loss: 6.5907e-04 - val_loss: 0.2548
Epoch 125/200
1/1 [==============================] - 0s 47ms/step - loss: 6.3417e-04 - val_loss: 0.2543
Epoch 126/200
1/1 [==============================] - 0s 48ms/step - loss: 6.1126e-04 - val_loss: 0.2538
Epoch 127/200
1/1 [==============================] - 0s 48ms/step - loss: 5.8679e-04 - val_loss: 0.2533
Epoch 128/200
1/1 [==============================] - 0s 46ms/step - loss: 5.64

Epoch 198/200
1/1 [==============================] - 0s 44ms/step - loss: 5.8288e-06 - val_loss: 0.2329
Epoch 199/200
1/1 [==============================] - 0s 49ms/step - loss: 5.2835e-06 - val_loss: 0.2329
Epoch 200/200
1/1 [==============================] - 0s 51ms/step - loss: 4.7747e-06 - val_loss: 0.2328
1/1 - 0s - loss: 0.1991 - 121ms/epoch - 121ms/step
Epoch 1/200
1/1 [==============================] - 1s 862ms/step - loss: 0.6970 - val_loss: 0.4192
Epoch 2/200
1/1 [==============================] - 0s 51ms/step - loss: 0.5964 - val_loss: 0.3685
Epoch 3/200
1/1 [==============================] - 0s 47ms/step - loss: 0.4945 - val_loss: 0.3237
Epoch 4/200
1/1 [==============================] - 0s 50ms/step - loss: 0.4033 - val_loss: 0.2856
Epoch 5/200
1/1 [==============================] - 0s 45ms/step - loss: 0.3281 - val_loss: 0.2547
Epoch 6/200
1/1 [==============================] - 0s 49ms/step - loss: 0.2700 - val_loss: 0.2303
Epoch 7/200
1/1 [==============================]

1/1 [==============================] - 0s 56ms/step - loss: 5.4031e-08 - val_loss: 0.0908
Epoch 159/200
1/1 [==============================] - 0s 49ms/step - loss: 4.3446e-08 - val_loss: 0.0908
Epoch 160/200
1/1 [==============================] - 0s 47ms/step - loss: 3.4131e-08 - val_loss: 0.0908
Epoch 161/200
1/1 [==============================] - 0s 47ms/step - loss: 2.5915e-08 - val_loss: 0.0908
Epoch 162/200
1/1 [==============================] - 0s 47ms/step - loss: 1.9095e-08 - val_loss: 0.0908
Epoch 163/200
1/1 [==============================] - 0s 46ms/step - loss: 1.3995e-08 - val_loss: 0.0908
Epoch 164/200
1/1 [==============================] - 0s 46ms/step - loss: 1.0582e-08 - val_loss: 0.0908
Epoch 165/200
1/1 [==============================] - 0s 50ms/step - loss: 8.3812e-09 - val_loss: 0.0908
Epoch 166/200
1/1 [==============================] - 0s 45ms/step - loss: 6.8726e-09 - val_loss: 0.0908
Epoch 167/200
1/1 [==============================] - 0s 50ms/step - loss: 5.80

1/1 [==============================] - 0s 52ms/step - loss: 2.2511e-07 - val_loss: 0.1701
Epoch 118/200
1/1 [==============================] - 0s 47ms/step - loss: 1.9862e-07 - val_loss: 0.1701
Epoch 119/200
1/1 [==============================] - 0s 47ms/step - loss: 1.7742e-07 - val_loss: 0.1701
Epoch 120/200
1/1 [==============================] - 0s 48ms/step - loss: 1.5620e-07 - val_loss: 0.1701
Epoch 121/200
1/1 [==============================] - 0s 45ms/step - loss: 1.3685e-07 - val_loss: 0.1701
Epoch 122/200
1/1 [==============================] - 0s 48ms/step - loss: 1.2436e-07 - val_loss: 0.1701
Epoch 123/200
1/1 [==============================] - 0s 47ms/step - loss: 1.2165e-07 - val_loss: 0.1701
Epoch 124/200
1/1 [==============================] - 0s 54ms/step - loss: 1.2679e-07 - val_loss: 0.1701
Epoch 125/200
1/1 [==============================] - 0s 58ms/step - loss: 1.3432e-07 - val_loss: 0.1701
Epoch 126/200
1/1 [==============================] - 0s 54ms/step - loss: 1.38

Epoch 196/200
1/1 [==============================] - 0s 50ms/step - loss: 8.7387e-11 - val_loss: 0.1701
Epoch 197/200
1/1 [==============================] - 0s 45ms/step - loss: 6.7786e-11 - val_loss: 0.1701
Epoch 198/200
1/1 [==============================] - 0s 48ms/step - loss: 4.9945e-11 - val_loss: 0.1701
Epoch 199/200
1/1 [==============================] - 0s 47ms/step - loss: 3.9268e-11 - val_loss: 0.1701
Epoch 200/200
1/1 [==============================] - 0s 46ms/step - loss: 3.4540e-11 - val_loss: 0.1701
1/1 - 0s - loss: 0.2910 - 109ms/epoch - 109ms/step
Fidelity between ideal and predicted Bloch vectors: 0.8654787353455238


## <font color='gold'>10 samples</font>

In [15]:
predictions_10 = [] # Collect the predictions in a list

for i in range(10): # for-loop for statistics (average on 10 runs)
    
    data = generate_Haar_data(1, 100) 
    density_matrix_noise_free = [*map(get_density_matrix, data)] 
    bloch_vectors_noise_free = [*map(Bloch_vector, density_matrix_noise_free)]

    density_matrix_with_noise = []
    for i in range(len(data)):
        single_data_with_noise = flip_error(1, density_matrix_noise_free[i], 'phase', 0.2)
        density_matrix_with_noise.append(single_data_with_noise)
    bloch_vectors_with_noise = [*map(Bloch_vector, density_matrix_with_noise)]
    
   
    # Training, Validation and Test set
    x_train_list, x_val_list, x_test_list = bloch_vectors_with_noise[:10], bloch_vectors_with_noise[50:90], bloch_vectors_with_noise[90:]
    y_train_list, y_val_list, y_test_list = bloch_vectors_noise_free[:10], bloch_vectors_noise_free[50:90], bloch_vectors_noise_free[90:]


    x_train = tf.convert_to_tensor(x_train_list)
    y_train = tf.convert_to_tensor(y_train_list)


    x_val = tf.convert_to_tensor(x_val_list)
    y_val = tf.convert_to_tensor(y_val_list)

    x_test = tf.convert_to_tensor(x_test_list)
    y_test = tf.convert_to_tensor(y_test_list)
    

    model = tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(3,)),
      tf.keras.layers.Dense(64, activation='relu'), 
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(3),
      tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
       ])
    
 
    loss_fn = tf.keras.losses.mse
    adam_opt = tf.optimizers.Adam(0.001)
    model.compile(optimizer=adam_opt, 
              loss=loss_fn)
    

    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=10, epochs=200)
    score = model.evaluate(x_test,  y_test, verbose=2)
    
    y_prediction = model(x_test)
    y_prediction = tf.cast(y_prediction, tf.complex128)
    y_test = tf.cast(y_test, tf.complex128)
    
    fidelities = [] # Collecting the reconstruction fidelities
    for i in range(len(y_prediction)):
        den_mat = density_matrix_from_bloch_vector(y_prediction[i])
        den_mat = np.asarray(den_mat)


        den_mat_id = density_matrix_from_bloch_vector(y_test[i])
        den_mat_id = np.asarray(den_mat_id)
        
        fidelity = fidelity_function(den_mat_id, den_mat)
        fidelities.append(fidelity)
    fid10_c = tf.math.reduce_mean(fidelities)
    fid10 = tf.cast(fid10_c, tf.float64)
    predictions_10.append(fid10)
print(f"Fidelity between ideal and predicted Bloch vectors: {tf.math.reduce_mean(predictions_10).numpy()}")

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 0.6015 - val_loss: 0.7099
Epoch 2/200
1/1 [==============================] - 0s 46ms/step - loss: 0.5030 - val_loss: 0.6616
Epoch 3/200
1/1 [==============================] - 0s 49ms/step - loss: 0.4244 - val_loss: 0.6176
Epoch 4/200
1/1 [==============================] - 0s 48ms/step - loss: 0.3551 - val_loss: 0.5735
Epoch 5/200
1/1 [==============================] - 0s 52ms/step - loss: 0.2891 - val_loss: 0.5244
Epoch 6/200
1/1 [==============================] - 0s 44ms/step - loss: 0.2261 - val_loss: 0.4650
Epoch 7/200
1/1 [==============================] - 0s 43ms/step - loss: 0.1705 - val_loss: 0.3988
Epoch 8/200
1/1 [==============================] - 0s 46ms/step - loss: 0.1261 - val_loss: 0.3435
Epoch 9/200
1/1 [==============================] - 0s 44ms/step - loss: 0.0956 - val_loss: 0.3003
Epoch 10/200
1/1 [==============================] - 0s 43ms/step - loss: 0.0786 - val_loss: 0.2667
Epoch 11/200
1/1 [===

1/1 [==============================] - 0s 42ms/step - loss: 1.8032e-07 - val_loss: 0.0779
Epoch 162/200
1/1 [==============================] - 0s 45ms/step - loss: 1.6816e-07 - val_loss: 0.0779
Epoch 163/200
1/1 [==============================] - 0s 43ms/step - loss: 1.5672e-07 - val_loss: 0.0779
Epoch 164/200
1/1 [==============================] - 0s 41ms/step - loss: 1.4589e-07 - val_loss: 0.0779
Epoch 165/200
1/1 [==============================] - 0s 44ms/step - loss: 1.3561e-07 - val_loss: 0.0779
Epoch 166/200
1/1 [==============================] - 0s 46ms/step - loss: 1.2583e-07 - val_loss: 0.0779
Epoch 167/200
1/1 [==============================] - 0s 43ms/step - loss: 1.1661e-07 - val_loss: 0.0779
Epoch 168/200
1/1 [==============================] - 0s 39ms/step - loss: 1.0799e-07 - val_loss: 0.0779
Epoch 169/200
1/1 [==============================] - 0s 42ms/step - loss: 1.0004e-07 - val_loss: 0.0779
Epoch 170/200
1/1 [==============================] - 0s 45ms/step - loss: 9.27

1/1 [==============================] - 0s 41ms/step - loss: 1.9538e-04 - val_loss: 0.0055
Epoch 122/200
1/1 [==============================] - 0s 46ms/step - loss: 1.9108e-04 - val_loss: 0.0055
Epoch 123/200
1/1 [==============================] - 0s 43ms/step - loss: 1.8687e-04 - val_loss: 0.0055
Epoch 124/200
1/1 [==============================] - 0s 42ms/step - loss: 1.8278e-04 - val_loss: 0.0055
Epoch 125/200
1/1 [==============================] - 0s 39ms/step - loss: 1.7881e-04 - val_loss: 0.0055
Epoch 126/200
1/1 [==============================] - 0s 45ms/step - loss: 1.7496e-04 - val_loss: 0.0055
Epoch 127/200
1/1 [==============================] - 0s 46ms/step - loss: 1.7122e-04 - val_loss: 0.0055
Epoch 128/200
1/1 [==============================] - 0s 42ms/step - loss: 1.6758e-04 - val_loss: 0.0055
Epoch 129/200
1/1 [==============================] - 0s 43ms/step - loss: 1.6404e-04 - val_loss: 0.0055
Epoch 130/200
1/1 [==============================] - 0s 44ms/step - loss: 1.60

Epoch 200/200
1/1 [==============================] - 0s 43ms/step - loss: 3.4349e-05 - val_loss: 0.0058
1/1 - 0s - loss: 0.0082 - 117ms/epoch - 117ms/step
Epoch 1/200
1/1 [==============================] - 1s 869ms/step - loss: 0.7126 - val_loss: 0.5661
Epoch 2/200
1/1 [==============================] - 0s 45ms/step - loss: 0.6588 - val_loss: 0.5151
Epoch 3/200
1/1 [==============================] - 0s 45ms/step - loss: 0.6177 - val_loss: 0.4721
Epoch 4/200
1/1 [==============================] - 0s 45ms/step - loss: 0.5836 - val_loss: 0.4345
Epoch 5/200
1/1 [==============================] - 0s 45ms/step - loss: 0.5520 - val_loss: 0.4007
Epoch 6/200
1/1 [==============================] - 0s 46ms/step - loss: 0.5190 - val_loss: 0.3698
Epoch 7/200
1/1 [==============================] - 0s 44ms/step - loss: 0.4846 - val_loss: 0.3419
Epoch 8/200
1/1 [==============================] - 0s 43ms/step - loss: 0.4498 - val_loss: 0.3164
Epoch 9/200
1/1 [==============================] - 0s 42ms/s

Epoch 82/200
1/1 [==============================] - 0s 39ms/step - loss: 3.8361e-04 - val_loss: 0.0277
Epoch 83/200
1/1 [==============================] - 0s 41ms/step - loss: 3.5893e-04 - val_loss: 0.0277
Epoch 84/200
1/1 [==============================] - 0s 43ms/step - loss: 3.4035e-04 - val_loss: 0.0278
Epoch 85/200
1/1 [==============================] - 0s 40ms/step - loss: 3.2692e-04 - val_loss: 0.0278
Epoch 86/200
1/1 [==============================] - 0s 44ms/step - loss: 3.1721e-04 - val_loss: 0.0278
Epoch 87/200
1/1 [==============================] - 0s 40ms/step - loss: 3.1018e-04 - val_loss: 0.0278
Epoch 88/200
1/1 [==============================] - 0s 40ms/step - loss: 3.0494e-04 - val_loss: 0.0277
Epoch 89/200
1/1 [==============================] - 0s 45ms/step - loss: 3.0073e-04 - val_loss: 0.0277
Epoch 90/200
1/1 [==============================] - 0s 44ms/step - loss: 2.9698e-04 - val_loss: 0.0276
Epoch 91/200
1/1 [==============================] - 0s 44ms/step - loss: 

...

1/1 [==============================] - 0s 44ms/step - loss: 4.3800e-05 - val_loss: 0.0107
Epoch 121/200
1/1 [==============================] - 0s 51ms/step - loss: 4.2255e-05 - val_loss: 0.0107
Epoch 122/200
1/1 [==============================] - 0s 41ms/step - loss: 4.0749e-05 - val_loss: 0.0107
Epoch 123/200
1/1 [==============================] - 0s 44ms/step - loss: 3.9289e-05 - val_loss: 0.0107
Epoch 124/200
1/1 [==============================] - 0s 52ms/step - loss: 3.7886e-05 - val_loss: 0.0107
Epoch 125/200
1/1 [==============================] - 0s 47ms/step - loss: 3.6570e-05 - val_loss: 0.0107
Epoch 126/200
1/1 [==============================] - 0s 48ms/step - loss: 3.5324e-05 - val_loss: 0.0106
Epoch 127/200
1/1 [==============================] - 0s 45ms/step - loss: 3.4167e-05 - val_loss: 0.0106
Epoch 128/200
1/1 [==============================] - 0s 46ms/step - loss: 3.3096e-05 - val_loss: 0.0106
Epoch 129/200
1/1 [==============================] - 0s 50ms/step - loss: 3.20

Epoch 199/200
1/1 [==============================] - 0s 43ms/step - loss: 4.3925e-06 - val_loss: 0.0100
Epoch 200/200
1/1 [==============================] - 0s 43ms/step - loss: 4.2594e-06 - val_loss: 0.0100
1/1 - 0s - loss: 0.0094 - 120ms/epoch - 120ms/step
Epoch 1/200
1/1 [==============================] - 1s 978ms/step - loss: 0.7918 - val_loss: 0.7492
Epoch 2/200
1/1 [==============================] - 0s 42ms/step - loss: 0.7123 - val_loss: 0.7129
Epoch 3/200
1/1 [==============================] - 0s 40ms/step - loss: 0.6405 - val_loss: 0.6801
Epoch 4/200
1/1 [==============================] - 0s 44ms/step - loss: 0.5777 - val_loss: 0.6510
Epoch 5/200
1/1 [==============================] - 0s 47ms/step - loss: 0.5234 - val_loss: 0.6250
Epoch 6/200
1/1 [==============================] - 0s 42ms/step - loss: 0.4772 - val_loss: 0.6013
Epoch 7/200
1/1 [==============================] - 0s 45ms/step - loss: 0.4381 - val_loss: 0.5789
Epoch 8/200
1/1 [==============================] - 0s 

1/1 [==============================] - 0s 41ms/step - loss: 4.4393e-06 - val_loss: 0.0139
Epoch 160/200
1/1 [==============================] - 0s 41ms/step - loss: 4.3336e-06 - val_loss: 0.0139
Epoch 161/200
1/1 [==============================] - 0s 41ms/step - loss: 4.2318e-06 - val_loss: 0.0139
Epoch 162/200
1/1 [==============================] - 0s 41ms/step - loss: 4.1336e-06 - val_loss: 0.0139
Epoch 163/200
1/1 [==============================] - 0s 41ms/step - loss: 4.0384e-06 - val_loss: 0.0139
Epoch 164/200
1/1 [==============================] - 0s 40ms/step - loss: 3.9457e-06 - val_loss: 0.0139
Epoch 165/200
1/1 [==============================] - 0s 41ms/step - loss: 3.8551e-06 - val_loss: 0.0139
Epoch 166/200
1/1 [==============================] - 0s 44ms/step - loss: 3.7664e-06 - val_loss: 0.0139
Epoch 167/200
1/1 [==============================] - 0s 41ms/step - loss: 3.6799e-06 - val_loss: 0.0139
Epoch 168/200
1/1 [==============================] - 0s 39ms/step - loss: 3.59

1/1 [==============================] - 0s 44ms/step - loss: 1.0234e-04 - val_loss: 0.0413
Epoch 120/200
1/1 [==============================] - 0s 43ms/step - loss: 1.0105e-04 - val_loss: 0.0412
Epoch 121/200
1/1 [==============================] - 0s 46ms/step - loss: 9.9804e-05 - val_loss: 0.0412
Epoch 122/200
1/1 [==============================] - 0s 44ms/step - loss: 9.8601e-05 - val_loss: 0.0411
Epoch 123/200
1/1 [==============================] - 0s 43ms/step - loss: 9.7435e-05 - val_loss: 0.0411
Epoch 124/200
1/1 [==============================] - 0s 41ms/step - loss: 9.6276e-05 - val_loss: 0.0411
Epoch 125/200
1/1 [==============================] - 0s 44ms/step - loss: 9.5117e-05 - val_loss: 0.0410
Epoch 126/200
1/1 [==============================] - 0s 42ms/step - loss: 9.3965e-05 - val_loss: 0.0410
Epoch 127/200
1/1 [==============================] - 0s 40ms/step - loss: 9.2806e-05 - val_loss: 0.0410
Epoch 128/200
1/1 [==============================] - 0s 45ms/step - loss: 9.16

Epoch 198/200
1/1 [==============================] - 0s 42ms/step - loss: 3.6914e-05 - val_loss: 0.0389
Epoch 199/200
1/1 [==============================] - 0s 43ms/step - loss: 3.6481e-05 - val_loss: 0.0389
Epoch 200/200
1/1 [==============================] - 0s 44ms/step - loss: 3.6050e-05 - val_loss: 0.0388
1/1 - 0s - loss: 0.0227 - 115ms/epoch - 115ms/step
Fidelity between ideal and predicted Bloch vectors: 0.9747512260918054


## <font color='gold'>20 samples</font>

In [17]:
predictions_20 = [] # Collect the predictions in a list

for i in range(10): # for-loop for statistics (average on 10 runs)
    data = generate_Haar_data(1, 100) 
    density_matrix_noise_free = [*map(get_density_matrix, data)] 
    bloch_vectors_noise_free = [*map(Bloch_vector, density_matrix_noise_free)]

    density_matrix_with_noise = []
    for i in range(len(data)):
        single_data_with_noise = flip_error(1, density_matrix_noise_free[i], 'phase', 0.2)
        density_matrix_with_noise.append(single_data_with_noise)
    bloch_vectors_with_noise = [*map(Bloch_vector, density_matrix_with_noise)]
    
   
    # Training, validation and test set
    x_train_list, x_val_list, x_test_list = bloch_vectors_with_noise[:20], bloch_vectors_with_noise[50:90], bloch_vectors_with_noise[90:]
    y_train_list, y_val_list, y_test_list = bloch_vectors_noise_free[:20], bloch_vectors_noise_free[50:90], bloch_vectors_noise_free[90:]


    x_train = tf.convert_to_tensor(x_train_list)
    y_train = tf.convert_to_tensor(y_train_list)


    x_val = tf.convert_to_tensor(x_val_list)
    y_val = tf.convert_to_tensor(y_val_list)

    x_test = tf.convert_to_tensor(x_test_list)
    y_test = tf.convert_to_tensor(y_test_list)
    

    model = tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(3,)),
      tf.keras.layers.Dense(64, activation='relu'), 
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(3),
      tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
       ])
    
 
    loss_fn = tf.keras.losses.mse
    adam_opt = tf.optimizers.Adam(0.001)
    model.compile(optimizer=adam_opt, 
              loss=loss_fn)
    

    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=5, epochs=200)
    score = model.evaluate(x_test,  y_test, verbose=2)
    
    y_prediction = model(x_test)
    y_prediction = tf.cast(y_prediction, tf.complex128)
    y_test = tf.cast(y_test, tf.complex128)
    
    fidelities = [] # Collecting the reconstruction fidelities
    for i in range(len(y_prediction)):
        den_mat = density_matrix_from_bloch_vector(y_prediction[i])
        den_mat = np.asarray(den_mat)


        den_mat_id = density_matrix_from_bloch_vector(y_test[i])
        den_mat_id = np.asarray(den_mat_id)
        
        fidelity = fidelity_function(den_mat_id, den_mat)
        fidelities.append(fidelity)
    fid20_c = tf.math.reduce_mean(fidelities)
    fid20 = tf.cast(fid20_c, tf.float64)
    predictions_20.append(fid20)
print(f"Fidelity between ideal and predicted Bloch vectors: {tf.math.reduce_mean(predictions_20).numpy()}")

Epoch 1/200
4/4 [==============================] - 1s 75ms/step - loss: 0.8991 - val_loss: 0.7077
Epoch 2/200
4/4 [==============================] - 0s 19ms/step - loss: 0.6239 - val_loss: 0.5150
Epoch 3/200
4/4 [==============================] - 0s 18ms/step - loss: 0.3587 - val_loss: 0.3413
Epoch 4/200
4/4 [==============================] - 0s 19ms/step - loss: 0.2060 - val_loss: 0.2403
Epoch 5/200
4/4 [==============================] - 0s 19ms/step - loss: 0.1355 - val_loss: 0.1772
Epoch 6/200
4/4 [==============================] - 0s 21ms/step - loss: 0.0896 - val_loss: 0.1197
Epoch 7/200
4/4 [==============================] - 0s 18ms/step - loss: 0.0639 - val_loss: 0.0687
Epoch 8/200
4/4 [==============================] - 0s 20ms/step - loss: 0.0403 - val_loss: 0.0405
Epoch 9/200
4/4 [==============================] - 0s 19ms/step - loss: 0.0281 - val_loss: 0.0264
Epoch 10/200
4/4 [==============================] - 0s 21ms/step - loss: 0.0187 - val_loss: 0.0216
Epoch 11/200
4/4 [=

4/4 [==============================] - 0s 16ms/step - loss: 5.2772e-06 - val_loss: 0.0030
Epoch 161/200
4/4 [==============================] - 0s 17ms/step - loss: 5.2545e-06 - val_loss: 0.0030
Epoch 162/200
4/4 [==============================] - 0s 17ms/step - loss: 4.9208e-06 - val_loss: 0.0030
Epoch 163/200
4/4 [==============================] - 0s 17ms/step - loss: 4.8991e-06 - val_loss: 0.0030
Epoch 164/200
4/4 [==============================] - 0s 17ms/step - loss: 4.8860e-06 - val_loss: 0.0030
Epoch 165/200
4/4 [==============================] - 0s 18ms/step - loss: 5.1316e-06 - val_loss: 0.0030
Epoch 166/200
4/4 [==============================] - 0s 18ms/step - loss: 5.2685e-06 - val_loss: 0.0030
Epoch 167/200
4/4 [==============================] - 0s 17ms/step - loss: 4.6544e-06 - val_loss: 0.0030
Epoch 168/200
4/4 [==============================] - 0s 17ms/step - loss: 4.7822e-06 - val_loss: 0.0030
Epoch 169/200
4/4 [==============================] - 0s 15ms/step - loss: 4.80

4/4 [==============================] - 0s 16ms/step - loss: 1.0491e-05 - val_loss: 0.0033
Epoch 119/200
4/4 [==============================] - 0s 16ms/step - loss: 1.0563e-05 - val_loss: 0.0032
Epoch 120/200
4/4 [==============================] - 0s 15ms/step - loss: 1.0279e-05 - val_loss: 0.0032
Epoch 121/200
4/4 [==============================] - 0s 16ms/step - loss: 9.6664e-06 - val_loss: 0.0032
Epoch 122/200
4/4 [==============================] - 0s 16ms/step - loss: 9.9459e-06 - val_loss: 0.0032
Epoch 123/200
4/4 [==============================] - 0s 16ms/step - loss: 9.5173e-06 - val_loss: 0.0032
Epoch 124/200
4/4 [==============================] - 0s 15ms/step - loss: 8.8347e-06 - val_loss: 0.0032
Epoch 125/200
4/4 [==============================] - 0s 16ms/step - loss: 8.4335e-06 - val_loss: 0.0032
Epoch 126/200
4/4 [==============================] - 0s 16ms/step - loss: 8.2455e-06 - val_loss: 0.0032
Epoch 127/200
4/4 [==============================] - 0s 17ms/step - loss: 7.84

Epoch 197/200
4/4 [==============================] - 0s 16ms/step - loss: 5.9822e-07 - val_loss: 0.0031
Epoch 198/200
4/4 [==============================] - 0s 16ms/step - loss: 6.1743e-07 - val_loss: 0.0031
Epoch 199/200
4/4 [==============================] - 0s 17ms/step - loss: 6.6074e-07 - val_loss: 0.0031
Epoch 200/200
4/4 [==============================] - 0s 19ms/step - loss: 6.0151e-07 - val_loss: 0.0031
1/1 - 0s - loss: 0.0028 - 142ms/epoch - 142ms/step
Epoch 1/200
4/4 [==============================] - 1s 63ms/step - loss: 0.4823 - val_loss: 0.4802
Epoch 2/200
4/4 [==============================] - 0s 18ms/step - loss: 0.3292 - val_loss: 0.3526
Epoch 3/200
4/4 [==============================] - 0s 18ms/step - loss: 0.1829 - val_loss: 0.2434
Epoch 4/200
4/4 [==============================] - 0s 18ms/step - loss: 0.1030 - val_loss: 0.1685
Epoch 5/200
4/4 [==============================] - 0s 18ms/step - loss: 0.0650 - val_loss: 0.1166
Epoch 6/200
4/4 [==========================

4/4 [==============================] - 0s 17ms/step - loss: 5.6405e-06 - val_loss: 0.0076
Epoch 156/200
4/4 [==============================] - 0s 18ms/step - loss: 6.5958e-06 - val_loss: 0.0076
Epoch 157/200
4/4 [==============================] - 0s 18ms/step - loss: 4.7538e-06 - val_loss: 0.0076
Epoch 158/200
4/4 [==============================] - 0s 20ms/step - loss: 4.8196e-06 - val_loss: 0.0076
Epoch 159/200
4/4 [==============================] - 0s 19ms/step - loss: 5.3939e-06 - val_loss: 0.0076
Epoch 160/200
4/4 [==============================] - 0s 18ms/step - loss: 6.2985e-06 - val_loss: 0.0076
Epoch 161/200
4/4 [==============================] - 0s 16ms/step - loss: 5.6254e-06 - val_loss: 0.0076
Epoch 162/200
4/4 [==============================] - 0s 17ms/step - loss: 6.1839e-06 - val_loss: 0.0076
Epoch 163/200
4/4 [==============================] - 0s 17ms/step - loss: 5.4944e-06 - val_loss: 0.0076
Epoch 164/200
4/4 [==============================] - 0s 17ms/step - loss: 5.53

...

4/4 [==============================] - 0s 17ms/step - loss: 2.5526e-06 - val_loss: 0.0066
Epoch 106/200
4/4 [==============================] - 0s 17ms/step - loss: 2.3700e-06 - val_loss: 0.0066
Epoch 107/200
4/4 [==============================] - 0s 16ms/step - loss: 2.4997e-06 - val_loss: 0.0066
Epoch 108/200
4/4 [==============================] - 0s 16ms/step - loss: 2.2419e-06 - val_loss: 0.0066
Epoch 109/200
4/4 [==============================] - 0s 17ms/step - loss: 2.0560e-06 - val_loss: 0.0066
Epoch 110/200
4/4 [==============================] - 0s 17ms/step - loss: 2.0057e-06 - val_loss: 0.0066
Epoch 111/200
4/4 [==============================] - 0s 17ms/step - loss: 2.1011e-06 - val_loss: 0.0066
Epoch 112/200
4/4 [==============================] - 0s 16ms/step - loss: 1.8210e-06 - val_loss: 0.0067
Epoch 113/200
4/4 [==============================] - 0s 16ms/step - loss: 1.9470e-06 - val_loss: 0.0067
Epoch 114/200
4/4 [==============================] - 0s 17ms/step - loss: 1.66

Epoch 184/200
4/4 [==============================] - 0s 19ms/step - loss: 5.6605e-07 - val_loss: 0.0067
Epoch 185/200
4/4 [==============================] - 0s 16ms/step - loss: 5.9612e-07 - val_loss: 0.0067
Epoch 186/200
4/4 [==============================] - 0s 18ms/step - loss: 6.1755e-07 - val_loss: 0.0067
Epoch 187/200
4/4 [==============================] - 0s 16ms/step - loss: 6.1650e-07 - val_loss: 0.0067
Epoch 188/200
4/4 [==============================] - 0s 24ms/step - loss: 5.6393e-07 - val_loss: 0.0067
Epoch 189/200
4/4 [==============================] - 0s 20ms/step - loss: 6.6603e-07 - val_loss: 0.0067
Epoch 190/200
4/4 [==============================] - 0s 18ms/step - loss: 4.1819e-07 - val_loss: 0.0067
Epoch 191/200
4/4 [==============================] - 0s 18ms/step - loss: 5.3663e-07 - val_loss: 0.0067
Epoch 192/200
4/4 [==============================] - 0s 19ms/step - loss: 7.8041e-07 - val_loss: 0.0067
Epoch 193/200
4/4 [==============================] - 0s 17ms/ste

4/4 [==============================] - 0s 18ms/step - loss: 5.3587e-06 - val_loss: 0.0381
Epoch 143/200
4/4 [==============================] - 0s 15ms/step - loss: 4.7545e-06 - val_loss: 0.0381
Epoch 144/200
4/4 [==============================] - 0s 18ms/step - loss: 4.7385e-06 - val_loss: 0.0380
Epoch 145/200
4/4 [==============================] - 0s 17ms/step - loss: 4.6195e-06 - val_loss: 0.0380
Epoch 146/200
4/4 [==============================] - 0s 17ms/step - loss: 4.3123e-06 - val_loss: 0.0380
Epoch 147/200
4/4 [==============================] - 0s 17ms/step - loss: 4.3517e-06 - val_loss: 0.0381
Epoch 148/200
4/4 [==============================] - 0s 16ms/step - loss: 4.3483e-06 - val_loss: 0.0381
Epoch 149/200
4/4 [==============================] - 0s 17ms/step - loss: 4.9211e-06 - val_loss: 0.0380
Epoch 150/200
4/4 [==============================] - 0s 17ms/step - loss: 4.1689e-06 - val_loss: 0.0380
Epoch 151/200
4/4 [==============================] - 0s 16ms/step - loss: 4.26

Epoch 101/200
4/4 [==============================] - 0s 17ms/step - loss: 6.0300e-06 - val_loss: 0.0069
Epoch 102/200
4/4 [==============================] - 0s 17ms/step - loss: 6.1221e-06 - val_loss: 0.0069
Epoch 103/200
4/4 [==============================] - 0s 18ms/step - loss: 7.0423e-06 - val_loss: 0.0068
Epoch 104/200
4/4 [==============================] - 0s 17ms/step - loss: 6.1325e-06 - val_loss: 0.0068
Epoch 105/200
4/4 [==============================] - 0s 16ms/step - loss: 5.3877e-06 - val_loss: 0.0069
Epoch 106/200
4/4 [==============================] - 0s 19ms/step - loss: 5.3793e-06 - val_loss: 0.0068
Epoch 107/200
4/4 [==============================] - 0s 17ms/step - loss: 5.3354e-06 - val_loss: 0.0069
Epoch 108/200
4/4 [==============================] - 0s 17ms/step - loss: 4.8446e-06 - val_loss: 0.0068
Epoch 109/200
4/4 [==============================] - 0s 17ms/step - loss: 4.5801e-06 - val_loss: 0.0068
Epoch 110/200
4/4 [==============================] - 0s 16ms/ste

Epoch 180/200
4/4 [==============================] - 0s 17ms/step - loss: 1.2686e-06 - val_loss: 0.0065
Epoch 181/200
4/4 [==============================] - 0s 16ms/step - loss: 1.5045e-06 - val_loss: 0.0065
Epoch 182/200
4/4 [==============================] - 0s 16ms/step - loss: 1.0793e-06 - val_loss: 0.0066
Epoch 183/200
4/4 [==============================] - 0s 17ms/step - loss: 1.1640e-06 - val_loss: 0.0066
Epoch 184/200
4/4 [==============================] - 0s 17ms/step - loss: 1.4943e-06 - val_loss: 0.0065
Epoch 185/200
4/4 [==============================] - 0s 16ms/step - loss: 1.5740e-06 - val_loss: 0.0066
Epoch 186/200
4/4 [==============================] - 0s 16ms/step - loss: 1.1668e-06 - val_loss: 0.0065
Epoch 187/200
4/4 [==============================] - 0s 16ms/step - loss: 1.6714e-06 - val_loss: 0.0065
Epoch 188/200
4/4 [==============================] - 0s 15ms/step - loss: 1.0660e-06 - val_loss: 0.0066
Epoch 189/200
4/4 [==============================] - 0s 16ms/ste

## <font color='gold'>30 samples</font>

In [18]:
predictions_30 = [] # Collect the predictions in a list

for i in range(10): # for-loop for statistics (average on 10 runs)
    data = generate_Haar_data(1, 100) 
    density_matrix_noise_free = [*map(get_density_matrix, data)] 
    bloch_vectors_noise_free = [*map(Bloch_vector, density_matrix_noise_free)]

    density_matrix_with_noise = []
    for i in range(len(data)):
        single_data_with_noise = flip_error(1, density_matrix_noise_free[i], 'phase', 0.2)
        density_matrix_with_noise.append(single_data_with_noise)
    bloch_vectors_with_noise = [*map(Bloch_vector, density_matrix_with_noise)]
    
   
    # Training, validation and test set
    x_train_list, x_val_list, x_test_list = bloch_vectors_with_noise[:30], bloch_vectors_with_noise[50:90], bloch_vectors_with_noise[90:]
    y_train_list, y_val_list, y_test_list = bloch_vectors_noise_free[:30], bloch_vectors_noise_free[50:90], bloch_vectors_noise_free[90:]


    x_train = tf.convert_to_tensor(x_train_list)
    y_train = tf.convert_to_tensor(y_train_list)


    x_val = tf.convert_to_tensor(x_val_list)
    y_val = tf.convert_to_tensor(y_val_list)

    x_test = tf.convert_to_tensor(x_test_list)
    y_test = tf.convert_to_tensor(y_test_list)
    

    model = tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(3,)),
      tf.keras.layers.Dense(64, activation='relu'), 
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(3),
      tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
       ])
    
 
    loss_fn = tf.keras.losses.mse
    adam_opt = tf.optimizers.Adam(0.001)
    model.compile(optimizer=adam_opt, 
              loss=loss_fn)
    

    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=6, epochs=200)
    score = model.evaluate(x_test,  y_test, verbose=2)
    
    y_prediction = model(x_test)
    y_prediction = tf.cast(y_prediction, tf.complex128)
    y_test = tf.cast(y_test, tf.complex128)
    
    fidelities = [] # Collecting the reconstruction fidelities
    for i in range(len(y_prediction)):
        den_mat = density_matrix_from_bloch_vector(y_prediction[i])
        den_mat = np.asarray(den_mat)


        den_mat_id = density_matrix_from_bloch_vector(y_test[i])
        den_mat_id = np.asarray(den_mat_id)
        
        fidelity = fidelity_function(den_mat_id, den_mat)
        fidelities.append(fidelity)
    fid30_c = tf.math.reduce_mean(fidelities)
    fid30 = tf.cast(fid30_c, tf.float64)
    predictions_30.append(fid30)
print(f"Fidelity between ideal and predicted Bloch vectors: {tf.math.reduce_mean(predictions_30).numpy()}")

Epoch 1/200
5/5 [==============================] - 1s 69ms/step - loss: 0.7937 - val_loss: 0.6375
Epoch 2/200
5/5 [==============================] - 0s 16ms/step - loss: 0.6095 - val_loss: 0.4853
Epoch 3/200
5/5 [==============================] - 0s 15ms/step - loss: 0.4503 - val_loss: 0.3445
Epoch 4/200
5/5 [==============================] - 0s 15ms/step - loss: 0.3076 - val_loss: 0.2120
Epoch 5/200
5/5 [==============================] - 0s 16ms/step - loss: 0.1691 - val_loss: 0.1134
Epoch 6/200
5/5 [==============================] - 0s 15ms/step - loss: 0.0968 - val_loss: 0.0685
Epoch 7/200
5/5 [==============================] - 0s 16ms/step - loss: 0.0548 - val_loss: 0.0461
Epoch 8/200
5/5 [==============================] - 0s 15ms/step - loss: 0.0302 - val_loss: 0.0323
Epoch 9/200
5/5 [==============================] - 0s 15ms/step - loss: 0.0174 - val_loss: 0.0245
Epoch 10/200
5/5 [==============================] - 0s 16ms/step - loss: 0.0125 - val_loss: 0.0188
Epoch 11/200
5/5 [=

5/5 [==============================] - 0s 14ms/step - loss: 4.0079e-06 - val_loss: 0.0023
Epoch 161/200
5/5 [==============================] - 0s 13ms/step - loss: 3.5381e-06 - val_loss: 0.0023
Epoch 162/200
5/5 [==============================] - 0s 13ms/step - loss: 3.8257e-06 - val_loss: 0.0023
Epoch 163/200
5/5 [==============================] - 0s 14ms/step - loss: 3.8631e-06 - val_loss: 0.0023
Epoch 164/200
5/5 [==============================] - 0s 13ms/step - loss: 3.5650e-06 - val_loss: 0.0023
Epoch 165/200
5/5 [==============================] - 0s 14ms/step - loss: 4.0822e-06 - val_loss: 0.0023
Epoch 166/200
5/5 [==============================] - 0s 13ms/step - loss: 3.3372e-06 - val_loss: 0.0023
Epoch 167/200
5/5 [==============================] - 0s 14ms/step - loss: 3.8230e-06 - val_loss: 0.0023
Epoch 168/200
5/5 [==============================] - 0s 14ms/step - loss: 3.1522e-06 - val_loss: 0.0023
Epoch 169/200
5/5 [==============================] - 0s 14ms/step - loss: 3.54

Epoch 40/200
5/5 [==============================] - 0s 14ms/step - loss: 2.6136e-04 - val_loss: 0.0010
Epoch 41/200
5/5 [==============================] - 0s 14ms/step - loss: 2.4748e-04 - val_loss: 9.6596e-04
Epoch 42/200
5/5 [==============================] - 0s 13ms/step - loss: 2.3233e-04 - val_loss: 9.3405e-04
Epoch 43/200
5/5 [==============================] - 0s 14ms/step - loss: 2.1879e-04 - val_loss: 9.2254e-04
Epoch 44/200
5/5 [==============================] - 0s 13ms/step - loss: 2.0709e-04 - val_loss: 9.0812e-04
Epoch 45/200
5/5 [==============================] - 0s 14ms/step - loss: 1.8939e-04 - val_loss: 8.9872e-04
Epoch 46/200
5/5 [==============================] - 0s 13ms/step - loss: 1.7643e-04 - val_loss: 8.9640e-04
Epoch 47/200
5/5 [==============================] - 0s 15ms/step - loss: 1.7204e-04 - val_loss: 8.7429e-04
Epoch 48/200
5/5 [==============================] - 0s 14ms/step - loss: 1.5988e-04 - val_loss: 8.6616e-04
Epoch 49/200
5/5 [=======================

Epoch 192/200
5/5 [==============================] - 0s 15ms/step - loss: 6.7934e-06 - val_loss: 6.9356e-04
Epoch 193/200
5/5 [==============================] - 0s 14ms/step - loss: 7.0571e-06 - val_loss: 6.9273e-04
Epoch 194/200
5/5 [==============================] - 0s 16ms/step - loss: 6.5317e-06 - val_loss: 6.9677e-04
Epoch 195/200
5/5 [==============================] - 0s 14ms/step - loss: 6.2883e-06 - val_loss: 6.9400e-04
Epoch 196/200
5/5 [==============================] - 0s 12ms/step - loss: 6.8308e-06 - val_loss: 6.9066e-04
Epoch 197/200
5/5 [==============================] - 0s 13ms/step - loss: 6.3075e-06 - val_loss: 6.9157e-04
Epoch 198/200
5/5 [==============================] - 0s 14ms/step - loss: 6.3733e-06 - val_loss: 6.9218e-04
Epoch 199/200
5/5 [==============================] - 0s 13ms/step - loss: 6.1801e-06 - val_loss: 6.9250e-04
Epoch 200/200
5/5 [==============================] - 0s 15ms/step - loss: 6.3382e-06 - val_loss: 6.9211e-04
1/1 - 0s - loss: 3.8878e-04 

5/5 [==============================] - 0s 15ms/step - loss: 6.8615e-06 - val_loss: 0.0080
Epoch 151/200
5/5 [==============================] - 0s 15ms/step - loss: 6.7783e-06 - val_loss: 0.0080
Epoch 152/200
5/5 [==============================] - 0s 15ms/step - loss: 5.9580e-06 - val_loss: 0.0080
Epoch 153/200
5/5 [==============================] - 0s 16ms/step - loss: 6.0528e-06 - val_loss: 0.0080
Epoch 154/200
5/5 [==============================] - 0s 18ms/step - loss: 5.5517e-06 - val_loss: 0.0080
Epoch 155/200
5/5 [==============================] - 0s 15ms/step - loss: 5.7557e-06 - val_loss: 0.0080
Epoch 156/200
5/5 [==============================] - 0s 14ms/step - loss: 5.8605e-06 - val_loss: 0.0080
Epoch 157/200
5/5 [==============================] - 0s 14ms/step - loss: 5.5492e-06 - val_loss: 0.0080
Epoch 158/200
5/5 [==============================] - 0s 14ms/step - loss: 5.2959e-06 - val_loss: 0.0080
Epoch 159/200
5/5 [==============================] - 0s 14ms/step - loss: 5.32

...

Epoch 99/200
5/5 [==============================] - 0s 13ms/step - loss: 1.4446e-05 - val_loss: 0.0029
Epoch 100/200
5/5 [==============================] - 0s 14ms/step - loss: 1.4250e-05 - val_loss: 0.0029
Epoch 101/200
5/5 [==============================] - 0s 13ms/step - loss: 1.4183e-05 - val_loss: 0.0029
Epoch 102/200
5/5 [==============================] - 0s 12ms/step - loss: 1.3696e-05 - val_loss: 0.0029
Epoch 103/200
5/5 [==============================] - 0s 14ms/step - loss: 1.2487e-05 - val_loss: 0.0029
Epoch 104/200
5/5 [==============================] - 0s 13ms/step - loss: 1.2344e-05 - val_loss: 0.0029
Epoch 105/200
5/5 [==============================] - 0s 13ms/step - loss: 1.2561e-05 - val_loss: 0.0029
Epoch 106/200
5/5 [==============================] - 0s 13ms/step - loss: 1.1974e-05 - val_loss: 0.0029
Epoch 107/200
5/5 [==============================] - 0s 14ms/step - loss: 1.1232e-05 - val_loss: 0.0029
Epoch 108/200
5/5 [==============================] - 0s 13ms/step

Epoch 178/200
5/5 [==============================] - 0s 14ms/step - loss: 3.7745e-06 - val_loss: 0.0028
Epoch 179/200
5/5 [==============================] - 0s 13ms/step - loss: 3.0141e-06 - val_loss: 0.0027
Epoch 180/200
5/5 [==============================] - 0s 14ms/step - loss: 4.0038e-06 - val_loss: 0.0027
Epoch 181/200
5/5 [==============================] - 0s 14ms/step - loss: 2.8674e-06 - val_loss: 0.0027
Epoch 182/200
5/5 [==============================] - 0s 13ms/step - loss: 3.6495e-06 - val_loss: 0.0027
Epoch 183/200
5/5 [==============================] - 0s 13ms/step - loss: 2.9849e-06 - val_loss: 0.0027
Epoch 184/200
5/5 [==============================] - 0s 13ms/step - loss: 2.7996e-06 - val_loss: 0.0027
Epoch 185/200
5/5 [==============================] - 0s 15ms/step - loss: 2.9023e-06 - val_loss: 0.0027
Epoch 186/200
5/5 [==============================] - 0s 15ms/step - loss: 2.6989e-06 - val_loss: 0.0027
Epoch 187/200
5/5 [==============================] - 0s 13ms/ste

5/5 [==============================] - 0s 13ms/step - loss: 5.0719e-06 - val_loss: 0.0023
Epoch 137/200
5/5 [==============================] - 0s 12ms/step - loss: 5.1575e-06 - val_loss: 0.0023
Epoch 138/200
5/5 [==============================] - 0s 13ms/step - loss: 5.2167e-06 - val_loss: 0.0022
Epoch 139/200
5/5 [==============================] - 0s 14ms/step - loss: 5.4268e-06 - val_loss: 0.0023
Epoch 140/200
5/5 [==============================] - 0s 14ms/step - loss: 4.7359e-06 - val_loss: 0.0022
Epoch 141/200
5/5 [==============================] - 0s 16ms/step - loss: 4.5098e-06 - val_loss: 0.0022
Epoch 142/200
5/5 [==============================] - 0s 22ms/step - loss: 4.4636e-06 - val_loss: 0.0022
Epoch 143/200
5/5 [==============================] - 0s 12ms/step - loss: 3.7921e-06 - val_loss: 0.0022
Epoch 144/200
5/5 [==============================] - 0s 14ms/step - loss: 4.0198e-06 - val_loss: 0.0022
Epoch 145/200
5/5 [==============================] - 0s 13ms/step - loss: 3.70

5/5 [==============================] - 0s 13ms/step - loss: 5.2019e-05 - val_loss: 0.0024
Epoch 96/200
5/5 [==============================] - 0s 13ms/step - loss: 4.3939e-05 - val_loss: 0.0024
Epoch 97/200
5/5 [==============================] - 0s 14ms/step - loss: 5.4847e-05 - val_loss: 0.0024
Epoch 98/200
5/5 [==============================] - 0s 14ms/step - loss: 4.3494e-05 - val_loss: 0.0024
Epoch 99/200
5/5 [==============================] - 0s 12ms/step - loss: 4.2684e-05 - val_loss: 0.0024
Epoch 100/200
5/5 [==============================] - 0s 14ms/step - loss: 4.5276e-05 - val_loss: 0.0024
Epoch 101/200
5/5 [==============================] - 0s 14ms/step - loss: 3.8043e-05 - val_loss: 0.0023
Epoch 102/200
5/5 [==============================] - 0s 13ms/step - loss: 3.7987e-05 - val_loss: 0.0023
Epoch 103/200
5/5 [==============================] - 0s 13ms/step - loss: 3.6814e-05 - val_loss: 0.0023
Epoch 104/200
5/5 [==============================] - 0s 13ms/step - loss: 3.9298e-

Epoch 174/200
5/5 [==============================] - 0s 13ms/step - loss: 1.2649e-06 - val_loss: 0.0022
Epoch 175/200
5/5 [==============================] - 0s 13ms/step - loss: 1.8124e-06 - val_loss: 0.0022
Epoch 176/200
5/5 [==============================] - 0s 13ms/step - loss: 1.3841e-06 - val_loss: 0.0022
Epoch 177/200
5/5 [==============================] - 0s 13ms/step - loss: 1.2164e-06 - val_loss: 0.0022
Epoch 178/200
5/5 [==============================] - 0s 13ms/step - loss: 1.1320e-06 - val_loss: 0.0022
Epoch 179/200
5/5 [==============================] - 0s 13ms/step - loss: 9.1427e-07 - val_loss: 0.0022
Epoch 180/200
5/5 [==============================] - 0s 14ms/step - loss: 1.0055e-06 - val_loss: 0.0022
Epoch 181/200
5/5 [==============================] - 0s 13ms/step - loss: 1.2571e-06 - val_loss: 0.0022
Epoch 182/200
5/5 [==============================] - 0s 16ms/step - loss: 9.1668e-07 - val_loss: 0.0022
Epoch 183/200
5/5 [==============================] - 0s 16ms/ste

## <font color='gold'>40 samples</font>

In [19]:
predictions_40 = [] # Collect the predictions in a list

for i in range(10): # for-loop for statistics (average on 10 runs)
    data = generate_Haar_data(1, 100) 
    density_matrix_noise_free = [*map(get_density_matrix, data)] 
    bloch_vectors_noise_free = [*map(Bloch_vector, density_matrix_noise_free)]

    density_matrix_with_noise = []
    for i in range(len(data)):
        single_data_with_noise = flip_error(1, density_matrix_noise_free[i], 'phase', 0.2)
        density_matrix_with_noise.append(single_data_with_noise)
    bloch_vectors_with_noise = [*map(Bloch_vector, density_matrix_with_noise)]
    
   
    # Training, validation and test set
    x_train_list, x_val_list, x_test_list = bloch_vectors_with_noise[:40], bloch_vectors_with_noise[50:90], bloch_vectors_with_noise[90:]
    y_train_list, y_val_list, y_test_list = bloch_vectors_noise_free[:40], bloch_vectors_noise_free[50:90], bloch_vectors_noise_free[90:]


    x_train = tf.convert_to_tensor(x_train_list)
    y_train = tf.convert_to_tensor(y_train_list)


    x_val = tf.convert_to_tensor(x_val_list)
    y_val = tf.convert_to_tensor(y_val_list)

    x_test = tf.convert_to_tensor(x_test_list)
    y_test = tf.convert_to_tensor(y_test_list)
    

    model = tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(3,)),
      tf.keras.layers.Dense(64, activation='relu'), 
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(3),
      
       ])
    
 
    loss_fn = tf.keras.losses.mse
    adam_opt = tf.optimizers.Adam(0.001)
    model.compile(optimizer=adam_opt, 
              loss=loss_fn)
    

    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=8, epochs=200)
    score = model.evaluate(x_test,  y_test, verbose=2)
    
    y_prediction = model(x_test)
    y_prediction = tf.cast(y_prediction, tf.complex128)
    y_test = tf.cast(y_test, tf.complex128)
    
    fidelities = [] # Collecting the reconstruction fidelities
    for i in range(len(y_prediction)):
        den_mat = density_matrix_from_bloch_vector(y_prediction[i])
        den_mat = np.asarray(den_mat)


        den_mat_id = density_matrix_from_bloch_vector(y_test[i])
        den_mat_id = np.asarray(den_mat_id)
        
        fidelity = fidelity_function(den_mat_id, den_mat)
        fidelities.append(fidelity)
    fid40_c = tf.math.reduce_mean(fidelities)
    fid40 = tf.cast(fid40_c, tf.float64)
    predictions_40.append(fid40)
print(f"Fidelity between ideal and predicted Bloch vectors: {tf.math.reduce_mean(predictions_40).numpy()}")

Epoch 1/200
5/5 [==============================] - 1s 52ms/step - loss: 0.3511 - val_loss: 0.3105
Epoch 2/200
5/5 [==============================] - 0s 13ms/step - loss: 0.2952 - val_loss: 0.2733
Epoch 3/200
5/5 [==============================] - 0s 13ms/step - loss: 0.2580 - val_loss: 0.2400
Epoch 4/200
5/5 [==============================] - 0s 14ms/step - loss: 0.2234 - val_loss: 0.2127
Epoch 5/200
5/5 [==============================] - 0s 13ms/step - loss: 0.1948 - val_loss: 0.1869
Epoch 6/200
5/5 [==============================] - 0s 13ms/step - loss: 0.1690 - val_loss: 0.1634
Epoch 7/200
5/5 [==============================] - 0s 13ms/step - loss: 0.1455 - val_loss: 0.1398
Epoch 8/200
5/5 [==============================] - 0s 14ms/step - loss: 0.1230 - val_loss: 0.1176
Epoch 9/200
5/5 [==============================] - 0s 13ms/step - loss: 0.1019 - val_loss: 0.0970
Epoch 10/200
5/5 [==============================] - 0s 13ms/step - loss: 0.0827 - val_loss: 0.0781
Epoch 11/200
5/5 [=

Epoch 80/200
5/5 [==============================] - 0s 13ms/step - loss: 4.0914e-05 - val_loss: 3.6668e-04
Epoch 81/200
5/5 [==============================] - 0s 11ms/step - loss: 3.9799e-05 - val_loss: 3.6614e-04
Epoch 82/200
5/5 [==============================] - 0s 12ms/step - loss: 3.8628e-05 - val_loss: 3.6305e-04
Epoch 83/200
5/5 [==============================] - 0s 12ms/step - loss: 3.8074e-05 - val_loss: 3.6612e-04
Epoch 84/200
5/5 [==============================] - 0s 13ms/step - loss: 3.7714e-05 - val_loss: 3.5751e-04
Epoch 85/200
5/5 [==============================] - 0s 12ms/step - loss: 3.6293e-05 - val_loss: 3.6001e-04
Epoch 86/200
5/5 [==============================] - 0s 12ms/step - loss: 3.5756e-05 - val_loss: 3.6104e-04
Epoch 87/200
5/5 [==============================] - 0s 12ms/step - loss: 3.4951e-05 - val_loss: 3.6399e-04
Epoch 88/200
5/5 [==============================] - 0s 12ms/step - loss: 3.4934e-05 - val_loss: 3.5468e-04
Epoch 89/200
5/5 [===================

5/5 [==============================] - 0s 12ms/step - loss: 1.0866e-05 - val_loss: 3.2401e-04
Epoch 157/200
5/5 [==============================] - 0s 14ms/step - loss: 1.1412e-05 - val_loss: 3.2257e-04
Epoch 158/200
5/5 [==============================] - 0s 12ms/step - loss: 1.1100e-05 - val_loss: 3.2291e-04
Epoch 159/200
5/5 [==============================] - 0s 13ms/step - loss: 1.1373e-05 - val_loss: 3.2514e-04
Epoch 160/200
5/5 [==============================] - 0s 12ms/step - loss: 1.1271e-05 - val_loss: 3.2194e-04
Epoch 161/200
5/5 [==============================] - 0s 12ms/step - loss: 1.0936e-05 - val_loss: 3.2435e-04
Epoch 162/200
5/5 [==============================] - 0s 12ms/step - loss: 1.0803e-05 - val_loss: 3.2558e-04
Epoch 163/200
5/5 [==============================] - 0s 13ms/step - loss: 1.1039e-05 - val_loss: 3.2144e-04
Epoch 164/200
5/5 [==============================] - 0s 12ms/step - loss: 1.0090e-05 - val_loss: 3.2349e-04
Epoch 165/200
5/5 [=======================

Epoch 34/200
5/5 [==============================] - 0s 13ms/step - loss: 3.3312e-04 - val_loss: 7.9963e-04
Epoch 35/200
5/5 [==============================] - 0s 12ms/step - loss: 3.1045e-04 - val_loss: 7.8497e-04
Epoch 36/200
5/5 [==============================] - 0s 12ms/step - loss: 2.8408e-04 - val_loss: 7.4552e-04
Epoch 37/200
5/5 [==============================] - 0s 12ms/step - loss: 2.6508e-04 - val_loss: 7.1130e-04
Epoch 38/200
5/5 [==============================] - 0s 13ms/step - loss: 2.4788e-04 - val_loss: 6.8665e-04
Epoch 39/200
5/5 [==============================] - 0s 12ms/step - loss: 2.3055e-04 - val_loss: 6.8462e-04
Epoch 40/200
5/5 [==============================] - 0s 13ms/step - loss: 2.1864e-04 - val_loss: 6.6435e-04
Epoch 41/200
5/5 [==============================] - 0s 13ms/step - loss: 2.0527e-04 - val_loss: 6.3515e-04
Epoch 42/200
5/5 [==============================] - 0s 12ms/step - loss: 1.9200e-04 - val_loss: 6.1600e-04
Epoch 43/200
5/5 [===================

Epoch 186/200
5/5 [==============================] - 0s 11ms/step - loss: 6.2248e-06 - val_loss: 3.4921e-04
Epoch 187/200
5/5 [==============================] - 0s 11ms/step - loss: 6.2856e-06 - val_loss: 3.4523e-04
Epoch 188/200
5/5 [==============================] - 0s 11ms/step - loss: 5.9093e-06 - val_loss: 3.5255e-04
Epoch 189/200
5/5 [==============================] - 0s 12ms/step - loss: 5.6621e-06 - val_loss: 3.4349e-04
Epoch 190/200
5/5 [==============================] - 0s 12ms/step - loss: 6.0380e-06 - val_loss: 3.5217e-04
Epoch 191/200
5/5 [==============================] - 0s 12ms/step - loss: 5.6289e-06 - val_loss: 3.4991e-04
Epoch 192/200
5/5 [==============================] - 0s 12ms/step - loss: 5.2906e-06 - val_loss: 3.4515e-04
Epoch 193/200
5/5 [==============================] - 0s 13ms/step - loss: 5.5617e-06 - val_loss: 3.4619e-04
Epoch 194/200
5/5 [==============================] - 0s 13ms/step - loss: 5.3713e-06 - val_loss: 3.4764e-04
Epoch 195/200
5/5 [=========

...

5/5 [==============================] - 0s 12ms/step - loss: 0.0072 - val_loss: 0.0135
Epoch 15/200
5/5 [==============================] - 0s 12ms/step - loss: 0.0053 - val_loss: 0.0104
Epoch 16/200
5/5 [==============================] - 0s 12ms/step - loss: 0.0043 - val_loss: 0.0080
Epoch 17/200
5/5 [==============================] - 0s 11ms/step - loss: 0.0033 - val_loss: 0.0064
Epoch 18/200
5/5 [==============================] - 0s 13ms/step - loss: 0.0026 - val_loss: 0.0050
Epoch 19/200
5/5 [==============================] - 0s 11ms/step - loss: 0.0020 - val_loss: 0.0044
Epoch 20/200
5/5 [==============================] - 0s 12ms/step - loss: 0.0015 - val_loss: 0.0038
Epoch 21/200
5/5 [==============================] - 0s 13ms/step - loss: 0.0012 - val_loss: 0.0035
Epoch 22/200
5/5 [==============================] - 0s 11ms/step - loss: 9.7337e-04 - val_loss: 0.0033
Epoch 23/200
5/5 [==============================] - 0s 12ms/step - loss: 7.7708e-04 - val_loss: 0.0030
Epoch 24/200
5/

5/5 [==============================] - 0s 12ms/step - loss: 2.6107e-05 - val_loss: 9.4299e-04
Epoch 94/200
5/5 [==============================] - 0s 12ms/step - loss: 2.4909e-05 - val_loss: 9.4663e-04
Epoch 95/200
5/5 [==============================] - 0s 12ms/step - loss: 2.6157e-05 - val_loss: 9.8133e-04
Epoch 96/200
5/5 [==============================] - 0s 12ms/step - loss: 2.3536e-05 - val_loss: 9.4533e-04
Epoch 97/200
5/5 [==============================] - 0s 12ms/step - loss: 2.4868e-05 - val_loss: 9.2932e-04
Epoch 98/200
5/5 [==============================] - 0s 11ms/step - loss: 2.1868e-05 - val_loss: 9.6354e-04
Epoch 99/200
5/5 [==============================] - 0s 11ms/step - loss: 2.3310e-05 - val_loss: 9.5513e-04
Epoch 100/200
5/5 [==============================] - 0s 12ms/step - loss: 2.0667e-05 - val_loss: 9.4706e-04
Epoch 101/200
5/5 [==============================] - 0s 11ms/step - loss: 2.3136e-05 - val_loss: 9.4532e-04
Epoch 102/200
5/5 [=============================

5/5 [==============================] - 0s 12ms/step - loss: 5.6287e-06 - val_loss: 9.2914e-04
Epoch 170/200
5/5 [==============================] - 0s 11ms/step - loss: 6.4933e-06 - val_loss: 9.2573e-04
Epoch 171/200
5/5 [==============================] - 0s 12ms/step - loss: 5.7969e-06 - val_loss: 9.1015e-04
Epoch 172/200
5/5 [==============================] - 0s 11ms/step - loss: 6.1025e-06 - val_loss: 9.2141e-04
Epoch 173/200
5/5 [==============================] - 0s 12ms/step - loss: 5.6956e-06 - val_loss: 9.2464e-04
Epoch 174/200
5/5 [==============================] - 0s 11ms/step - loss: 5.3917e-06 - val_loss: 9.2705e-04
Epoch 175/200
5/5 [==============================] - 0s 11ms/step - loss: 5.0893e-06 - val_loss: 9.0726e-04
Epoch 176/200
5/5 [==============================] - 0s 12ms/step - loss: 5.2505e-06 - val_loss: 9.0827e-04
Epoch 177/200
5/5 [==============================] - 0s 12ms/step - loss: 5.3853e-06 - val_loss: 9.4057e-04
Epoch 178/200
5/5 [=======================

5/5 [==============================] - 0s 12ms/step - loss: 1.8672e-04 - val_loss: 8.0858e-04
Epoch 48/200
5/5 [==============================] - 0s 12ms/step - loss: 1.7853e-04 - val_loss: 7.9037e-04
Epoch 49/200
5/5 [==============================] - 0s 12ms/step - loss: 1.7128e-04 - val_loss: 7.8215e-04
Epoch 50/200
5/5 [==============================] - 0s 13ms/step - loss: 1.6704e-04 - val_loss: 7.7262e-04
Epoch 51/200
5/5 [==============================] - 0s 12ms/step - loss: 1.6671e-04 - val_loss: 7.5110e-04
Epoch 52/200
5/5 [==============================] - 0s 12ms/step - loss: 1.6085e-04 - val_loss: 7.4956e-04
Epoch 53/200
5/5 [==============================] - 0s 12ms/step - loss: 1.5344e-04 - val_loss: 7.5239e-04
Epoch 54/200
5/5 [==============================] - 0s 12ms/step - loss: 1.4765e-04 - val_loss: 7.3219e-04
Epoch 55/200
5/5 [==============================] - 0s 12ms/step - loss: 1.4236e-04 - val_loss: 7.2926e-04
Epoch 56/200
5/5 [==============================] 

Epoch 199/200
5/5 [==============================] - 0s 11ms/step - loss: 1.6811e-05 - val_loss: 4.1574e-04
Epoch 200/200
5/5 [==============================] - 0s 13ms/step - loss: 1.5269e-05 - val_loss: 4.1039e-04
1/1 - 0s - loss: 6.3071e-04 - 111ms/epoch - 111ms/step
Fidelity between ideal and predicted Bloch vectors: 0.9945558486774624


# **<font color='orange'> Training with Infidelity</font>**

## <font color='gold'>5 samples</font>

In [21]:
inf_predictions_5 = [] # Collecting the predictions in a list

for i in range(10): # for-loop for statistics (average on 10 runs)
    data = generate_Haar_data(1, 100) 
    density_matrix_noise_free = [*map(get_density_matrix, data)] 
    bloch_vectors_noise_free = [*map(Bloch_vector, density_matrix_noise_free)]

    density_matrix_with_noise = []
    for i in range(len(data)):
        single_data_with_noise = flip_error(1, density_matrix_noise_free[i], 'phase', 0.2)
        density_matrix_with_noise.append(single_data_with_noise)
    bloch_vectors_with_noise = [*map(Bloch_vector, density_matrix_with_noise)]
    
   
    # Training, validation and test set
    x_train_list, x_val_list, x_test_list = bloch_vectors_with_noise[:5], bloch_vectors_with_noise[50:90], bloch_vectors_with_noise[90:]
    y_train_list, y_val_list, y_test_list = bloch_vectors_noise_free[:5], bloch_vectors_noise_free[50:90], bloch_vectors_noise_free[90:]


    x_train = tf.convert_to_tensor(x_train_list)
    y_train = tf.convert_to_tensor(y_train_list)


    x_val = tf.convert_to_tensor(x_val_list)
    y_val = tf.convert_to_tensor(y_val_list)

    x_test = tf.convert_to_tensor(x_test_list)
    y_test = tf.convert_to_tensor(y_test_list)
    

    model = tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(3,)),
      tf.keras.layers.Dense(64, activation='relu'), 
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(3),
      tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
       ])
    

    loss_fn = tf.keras.losses.mse
    adam_opt = tf.optimizers.Adam(0.001)
    model.compile(optimizer=adam_opt, 
              loss=infidelity1)

    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=5, epochs=200)
    score = model.evaluate(x_test,  y_test, verbose=2)
    
    y_prediction = model(x_test)
    y_prediction = tf.cast(y_prediction, tf.complex128)
    y_test = tf.cast(y_test, tf.complex128)
    
    fidelities = [] # Collecting the reconstruction fidelities
    for i in range(len(y_prediction)):
        den_mat = density_matrix_from_bloch_vector(y_prediction[i])
        den_mat = np.asarray(den_mat)


        den_mat_id = density_matrix_from_bloch_vector(y_test[i])
        den_mat_id = np.asarray(den_mat_id)
        
        fidelity = fidelity_function(den_mat_id, den_mat)
        fidelities.append(fidelity)
    inf_fid5_c = tf.math.reduce_mean(fidelities)
    inf_fid5 = tf.cast(inf_fid5_c, tf.float64)
    inf_predictions_5.append(inf_fid5)
print(f"Fidelity between ideal and predicted Bloch vectors: {tf.math.reduce_mean(inf_predictions_5).numpy()}")

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 0.7069 - val_loss: 0.4877
Epoch 2/200
1/1 [==============================] - 0s 57ms/step - loss: 0.6228 - val_loss: 0.4656
Epoch 3/200
1/1 [==============================] - 0s 56ms/step - loss: 0.5528 - val_loss: 0.4467
Epoch 4/200
1/1 [==============================] - 0s 56ms/step - loss: 0.4940 - val_loss: 0.4304
Epoch 5/200
1/1 [==============================] - 0s 52ms/step - loss: 0.4455 - val_loss: 0.4155
Epoch 6/200
1/1 [==============================] - 0s 56ms/step - loss: 0.4033 - val_loss: 0.4021
Epoch 7/200
1/1 [==============================] - 0s 59ms/step - loss: 0.3660 - val_loss: 0.3899
Epoch 8/200
1/1 [==============================] - 0s 68ms/step - loss: 0.3341 - val_loss: 0.3782
Epoch 9/200
1/1 [==============================] - 0s 63ms/step - loss: 0.3059 - val_loss: 0.3668
Epoch 10/200
1/1 [==============================] - 0s 56ms/step - loss: 0.2799 - val_loss: 0.3557
Epoch 11/200
1/1 [===

1/1 [==============================] - 0s 51ms/step - loss: 2.3842e-08 - val_loss: 0.0660
Epoch 162/200
1/1 [==============================] - 0s 56ms/step - loss: 1.1921e-08 - val_loss: 0.0661
Epoch 163/200
1/1 [==============================] - 0s 49ms/step - loss: 3.5763e-08 - val_loss: 0.0661
Epoch 164/200
1/1 [==============================] - 0s 47ms/step - loss: 5.9605e-08 - val_loss: 0.0661
Epoch 165/200
1/1 [==============================] - 0s 45ms/step - loss: 0.0000e+00 - val_loss: 0.0661
Epoch 166/200
1/1 [==============================] - 0s 57ms/step - loss: 3.5763e-08 - val_loss: 0.0661
Epoch 167/200
1/1 [==============================] - 0s 54ms/step - loss: 0.0000e+00 - val_loss: 0.0661
Epoch 168/200
1/1 [==============================] - 0s 52ms/step - loss: -2.3842e-08 - val_loss: 0.0661
Epoch 169/200
1/1 [==============================] - 0s 48ms/step - loss: 3.5763e-08 - val_loss: 0.0661
Epoch 170/200
1/1 [==============================] - 0s 46ms/step - loss: 1.1

1/1 [==============================] - 0s 49ms/step - loss: 5.8413e-07 - val_loss: 0.0857
Epoch 122/200
1/1 [==============================] - 0s 46ms/step - loss: 4.5300e-07 - val_loss: 0.0857
Epoch 123/200
1/1 [==============================] - 0s 52ms/step - loss: 3.9339e-07 - val_loss: 0.0858
Epoch 124/200
1/1 [==============================] - 0s 51ms/step - loss: 3.3379e-07 - val_loss: 0.0858
Epoch 125/200
1/1 [==============================] - 0s 51ms/step - loss: 3.2187e-07 - val_loss: 0.0858
Epoch 126/200
1/1 [==============================] - 0s 49ms/step - loss: 2.9802e-07 - val_loss: 0.0858
Epoch 127/200
1/1 [==============================] - 0s 54ms/step - loss: 2.8610e-07 - val_loss: 0.0858
Epoch 128/200
1/1 [==============================] - 0s 48ms/step - loss: 2.9802e-07 - val_loss: 0.0858
Epoch 129/200
1/1 [==============================] - 0s 49ms/step - loss: 2.8610e-07 - val_loss: 0.0858
Epoch 130/200
1/1 [==============================] - 0s 53ms/step - loss: 2.74

Epoch 200/200
1/1 [==============================] - 0s 44ms/step - loss: 0.0000e+00 - val_loss: 0.0856
1/1 - 0s - loss: 0.0655 - 134ms/epoch - 134ms/step
Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 0.1376 - val_loss: 0.2026
Epoch 2/200
1/1 [==============================] - 0s 49ms/step - loss: 0.0486 - val_loss: 0.1934
Epoch 3/200
1/1 [==============================] - 0s 53ms/step - loss: 0.0203 - val_loss: 0.1953
Epoch 4/200
1/1 [==============================] - 0s 50ms/step - loss: 0.0124 - val_loss: 0.1997
Epoch 5/200
1/1 [==============================] - 0s 54ms/step - loss: 0.0115 - val_loss: 0.2021
Epoch 6/200
1/1 [==============================] - 0s 56ms/step - loss: 0.0115 - val_loss: 0.2015
Epoch 7/200
1/1 [==============================] - 0s 55ms/step - loss: 0.0107 - val_loss: 0.1987
Epoch 8/200
1/1 [==============================] - 0s 61ms/step - loss: 0.0093 - val_loss: 0.1950
Epoch 9/200
1/1 [==============================] - 0s 58ms/step

Epoch 80/200
1/1 [==============================] - 0s 48ms/step - loss: 2.8014e-06 - val_loss: 0.1306
Epoch 81/200
1/1 [==============================] - 0s 47ms/step - loss: 2.5868e-06 - val_loss: 0.1307
Epoch 82/200
1/1 [==============================] - 0s 48ms/step - loss: 2.2411e-06 - val_loss: 0.1308
Epoch 83/200
1/1 [==============================] - 0s 49ms/step - loss: 1.9073e-06 - val_loss: 0.1310
Epoch 84/200
1/1 [==============================] - 0s 52ms/step - loss: 1.5855e-06 - val_loss: 0.1311
Epoch 85/200
1/1 [==============================] - 0s 56ms/step - loss: 1.3351e-06 - val_loss: 0.1313
Epoch 86/200
1/1 [==============================] - 0s 53ms/step - loss: 1.1325e-06 - val_loss: 0.1314
Epoch 87/200
1/1 [==============================] - 0s 53ms/step - loss: 1.0014e-06 - val_loss: 0.1315
Epoch 88/200
1/1 [==============================] - 0s 52ms/step - loss: 9.5367e-07 - val_loss: 0.1316
Epoch 89/200
1/1 [==============================] - 0s 54ms/step - loss: 

...

Epoch 118/200
1/1 [==============================] - 0s 50ms/step - loss: 3.3498e-06 - val_loss: 0.2772
Epoch 119/200
1/1 [==============================] - 0s 48ms/step - loss: 2.9206e-06 - val_loss: 0.2770
Epoch 120/200
1/1 [==============================] - 0s 51ms/step - loss: 2.5034e-06 - val_loss: 0.2769
Epoch 121/200
1/1 [==============================] - 0s 53ms/step - loss: 2.1815e-06 - val_loss: 0.2768
Epoch 122/200
1/1 [==============================] - 0s 47ms/step - loss: 1.8716e-06 - val_loss: 0.2767
Epoch 123/200
1/1 [==============================] - 0s 57ms/step - loss: 1.5974e-06 - val_loss: 0.2765
Epoch 124/200
1/1 [==============================] - 0s 47ms/step - loss: 1.3947e-06 - val_loss: 0.2764
Epoch 125/200
1/1 [==============================] - 0s 50ms/step - loss: 1.2159e-06 - val_loss: 0.2763
Epoch 126/200
1/1 [==============================] - 0s 48ms/step - loss: 1.0252e-06 - val_loss: 0.2762
Epoch 127/200
1/1 [==============================] - 0s 51ms/ste

Epoch 77/200
1/1 [==============================] - 0s 56ms/step - loss: 1.1719e-04 - val_loss: 0.0955
Epoch 78/200
1/1 [==============================] - 0s 54ms/step - loss: 1.0927e-04 - val_loss: 0.0955
Epoch 79/200
1/1 [==============================] - 0s 59ms/step - loss: 1.0087e-04 - val_loss: 0.0954
Epoch 80/200
1/1 [==============================] - 0s 64ms/step - loss: 9.2459e-05 - val_loss: 0.0953
Epoch 81/200
1/1 [==============================] - 0s 58ms/step - loss: 8.4710e-05 - val_loss: 0.0953
Epoch 82/200
1/1 [==============================] - 0s 50ms/step - loss: 7.7975e-05 - val_loss: 0.0952
Epoch 83/200
1/1 [==============================] - 0s 49ms/step - loss: 7.2467e-05 - val_loss: 0.0951
Epoch 84/200
1/1 [==============================] - 0s 53ms/step - loss: 6.8033e-05 - val_loss: 0.0951
Epoch 85/200
1/1 [==============================] - 0s 51ms/step - loss: 6.4337e-05 - val_loss: 0.0951
Epoch 86/200
1/1 [==============================] - 0s 52ms/step - loss: 

Epoch 156/200
1/1 [==============================] - 0s 55ms/step - loss: 2.3842e-08 - val_loss: 0.0943
Epoch 157/200
1/1 [==============================] - 0s 51ms/step - loss: 2.3842e-08 - val_loss: 0.0943
Epoch 158/200
1/1 [==============================] - 0s 56ms/step - loss: 0.0000e+00 - val_loss: 0.0943
Epoch 159/200
1/1 [==============================] - 0s 53ms/step - loss: 2.3842e-08 - val_loss: 0.0943
Epoch 160/200
1/1 [==============================] - 0s 52ms/step - loss: 1.1921e-08 - val_loss: 0.0943
Epoch 161/200
1/1 [==============================] - 0s 55ms/step - loss: -1.1921e-08 - val_loss: 0.0943
Epoch 162/200
1/1 [==============================] - 0s 54ms/step - loss: 7.1526e-08 - val_loss: 0.0943
Epoch 163/200
1/1 [==============================] - 0s 53ms/step - loss: 1.1921e-08 - val_loss: 0.0943
Epoch 164/200
1/1 [==============================] - 0s 52ms/step - loss: 2.3842e-08 - val_loss: 0.0943
Epoch 165/200
1/1 [==============================] - 0s 47ms/st

Epoch 36/200
1/1 [==============================] - 0s 48ms/step - loss: 3.0773e-04 - val_loss: 0.1712
Epoch 37/200
1/1 [==============================] - 0s 48ms/step - loss: 2.2269e-04 - val_loss: 0.1697
Epoch 38/200
1/1 [==============================] - 0s 47ms/step - loss: 1.6109e-04 - val_loss: 0.1684
Epoch 39/200
1/1 [==============================] - 0s 51ms/step - loss: 1.2263e-04 - val_loss: 0.1672
Epoch 40/200
1/1 [==============================] - 0s 50ms/step - loss: 1.0244e-04 - val_loss: 0.1662
Epoch 41/200
1/1 [==============================] - 0s 54ms/step - loss: 9.3734e-05 - val_loss: 0.1653
Epoch 42/200
1/1 [==============================] - 0s 50ms/step - loss: 9.0075e-05 - val_loss: 0.1647
Epoch 43/200
1/1 [==============================] - 0s 55ms/step - loss: 8.6248e-05 - val_loss: 0.1642
Epoch 44/200
1/1 [==============================] - 0s 53ms/step - loss: 7.9775e-05 - val_loss: 0.1639
Epoch 45/200
1/1 [==============================] - 0s 49ms/step - loss: 

Epoch 194/200
1/1 [==============================] - 0s 48ms/step - loss: 3.5763e-08 - val_loss: 0.1661
Epoch 195/200
1/1 [==============================] - 0s 48ms/step - loss: 1.1921e-08 - val_loss: 0.1661
Epoch 196/200
1/1 [==============================] - 0s 48ms/step - loss: 2.3842e-08 - val_loss: 0.1661
Epoch 197/200
1/1 [==============================] - 0s 51ms/step - loss: -2.3842e-08 - val_loss: 0.1661
Epoch 198/200
1/1 [==============================] - 0s 48ms/step - loss: 0.0000e+00 - val_loss: 0.1661
Epoch 199/200
1/1 [==============================] - 0s 46ms/step - loss: 0.0000e+00 - val_loss: 0.1661
Epoch 200/200
1/1 [==============================] - 0s 48ms/step - loss: 0.0000e+00 - val_loss: 0.1661
1/1 - 0s - loss: 0.1585 - 129ms/epoch - 129ms/step
Fidelity between ideal and predicted Bloch vectors: 0.8903714306486531


## <font color='gold'>10 samples</font>

In [20]:
inf_predictions_10 = [] # Collecting the predictions in a list

for i in range(10): # for-loop for statistics (average on 10 runs)
    data = generate_Haar_data(1, 100) 
    density_matrix_noise_free = [*map(get_density_matrix, data)] 
    bloch_vectors_noise_free = [*map(Bloch_vector, density_matrix_noise_free)]

    density_matrix_with_noise = []
    for i in range(len(data)):
        single_data_with_noise = flip_error(1, density_matrix_noise_free[i], 'phase', 0.2)
        density_matrix_with_noise.append(single_data_with_noise)
    bloch_vectors_with_noise = [*map(Bloch_vector, density_matrix_with_noise)]
    
   
    # Training, validation and test set
    x_train_list, x_val_list, x_test_list = bloch_vectors_with_noise[:10], bloch_vectors_with_noise[50:90], bloch_vectors_with_noise[90:]
    y_train_list, y_val_list, y_test_list = bloch_vectors_noise_free[:10], bloch_vectors_noise_free[50:90], bloch_vectors_noise_free[90:]


    x_train = tf.convert_to_tensor(x_train_list)
    y_train = tf.convert_to_tensor(y_train_list)


    x_val = tf.convert_to_tensor(x_val_list)
    y_val = tf.convert_to_tensor(y_val_list)

    x_test = tf.convert_to_tensor(x_test_list)
    y_test = tf.convert_to_tensor(y_test_list)
    

    model = tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(3,)),
      tf.keras.layers.Dense(64, activation='relu'), 
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(3),
      tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
       ])
    

    loss_fn = tf.keras.losses.mse
    adam_opt = tf.optimizers.Adam(0.001)
    model.compile(optimizer=adam_opt, 
              loss=infidelity1)

    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=10, epochs=200)
    score = model.evaluate(x_test,  y_test, verbose=2)
    
    y_prediction = model(x_test)
    y_prediction = tf.cast(y_prediction, tf.complex128)
    y_test = tf.cast(y_test, tf.complex128)
    
    fidelities = [] # Collecting the reconstruction fidelities
    for i in range(len(y_prediction)):
        den_mat = density_matrix_from_bloch_vector(y_prediction[i])
        den_mat = np.asarray(den_mat)


        den_mat_id = density_matrix_from_bloch_vector(y_test[i])
        den_mat_id = np.asarray(den_mat_id)
        
        fidelity = fidelity_function(den_mat_id, den_mat)
        fidelities.append(fidelity)
    inf_fid10_c = tf.math.reduce_mean(fidelities)
    inf_fid10 = tf.cast(inf_fid10_c, tf.float64)
    inf_predictions_10.append(inf_fid10)
print(f"Fidelity between ideal and predicted Bloch vectors: {tf.math.reduce_mean(inf_predictions_10).numpy()}")

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 0.6582 - val_loss: 0.5118
Epoch 2/200
1/1 [==============================] - 0s 50ms/step - loss: 0.5475 - val_loss: 0.4603
Epoch 3/200
1/1 [==============================] - 0s 47ms/step - loss: 0.4515 - val_loss: 0.4079
Epoch 4/200
1/1 [==============================] - 0s 49ms/step - loss: 0.3723 - val_loss: 0.3572
Epoch 5/200
1/1 [==============================] - 0s 43ms/step - loss: 0.3052 - val_loss: 0.3022
Epoch 6/200
1/1 [==============================] - 0s 42ms/step - loss: 0.2462 - val_loss: 0.2492
Epoch 7/200
1/1 [==============================] - 0s 55ms/step - loss: 0.1956 - val_loss: 0.2094
Epoch 8/200
1/1 [==============================] - 0s 47ms/step - loss: 0.1533 - val_loss: 0.1757
Epoch 9/200
1/1 [==============================] - 0s 43ms/step - loss: 0.1196 - val_loss: 0.1493
Epoch 10/200
1/1 [==============================] - 0s 44ms/step - loss: 0.0952 - val_loss: 0.1302
Epoch 11/200
1/1 [===

1/1 [==============================] - 0s 43ms/step - loss: 5.4437e-05 - val_loss: 0.0108
Epoch 162/200
1/1 [==============================] - 0s 50ms/step - loss: 5.4204e-05 - val_loss: 0.0108
Epoch 163/200
1/1 [==============================] - 0s 42ms/step - loss: 5.3948e-05 - val_loss: 0.0108
Epoch 164/200
1/1 [==============================] - 0s 45ms/step - loss: 5.3710e-05 - val_loss: 0.0108
Epoch 165/200
1/1 [==============================] - 0s 44ms/step - loss: 5.3465e-05 - val_loss: 0.0108
Epoch 166/200
1/1 [==============================] - 0s 42ms/step - loss: 5.3209e-05 - val_loss: 0.0108
Epoch 167/200
1/1 [==============================] - 0s 45ms/step - loss: 5.2971e-05 - val_loss: 0.0107
Epoch 168/200
1/1 [==============================] - 0s 43ms/step - loss: 5.2756e-05 - val_loss: 0.0107
Epoch 169/200
1/1 [==============================] - 0s 45ms/step - loss: 5.2518e-05 - val_loss: 0.0107
Epoch 170/200
1/1 [==============================] - 0s 45ms/step - loss: 5.22

1/1 [==============================] - 0s 44ms/step - loss: 2.2298e-04 - val_loss: 0.0424
Epoch 123/200
1/1 [==============================] - 0s 42ms/step - loss: 2.1576e-04 - val_loss: 0.0423
Epoch 124/200
1/1 [==============================] - 0s 44ms/step - loss: 2.0873e-04 - val_loss: 0.0422
Epoch 125/200
1/1 [==============================] - 0s 40ms/step - loss: 2.0187e-04 - val_loss: 0.0421
Epoch 126/200
1/1 [==============================] - 0s 42ms/step - loss: 1.9524e-04 - val_loss: 0.0421
Epoch 127/200
1/1 [==============================] - 0s 44ms/step - loss: 1.8888e-04 - val_loss: 0.0420
Epoch 128/200
1/1 [==============================] - 0s 44ms/step - loss: 1.8269e-04 - val_loss: 0.0419
Epoch 129/200
1/1 [==============================] - 0s 40ms/step - loss: 1.7669e-04 - val_loss: 0.0418
Epoch 130/200
1/1 [==============================] - 0s 39ms/step - loss: 1.7092e-04 - val_loss: 0.0417
Epoch 131/200
1/1 [==============================] - 0s 42ms/step - loss: 1.65

1/1 - 0s - loss: 0.0436 - 132ms/epoch - 132ms/step
Epoch 1/200
1/1 [==============================] - 1s 901ms/step - loss: 0.3104 - val_loss: 0.2983
Epoch 2/200
1/1 [==============================] - 0s 41ms/step - loss: 0.2198 - val_loss: 0.2181
Epoch 3/200
1/1 [==============================] - 0s 43ms/step - loss: 0.1524 - val_loss: 0.1823
Epoch 4/200
1/1 [==============================] - 0s 45ms/step - loss: 0.1101 - val_loss: 0.1497
Epoch 5/200
1/1 [==============================] - 0s 44ms/step - loss: 0.0819 - val_loss: 0.1301
Epoch 6/200
1/1 [==============================] - 0s 43ms/step - loss: 0.0630 - val_loss: 0.1167
Epoch 7/200
1/1 [==============================] - 0s 44ms/step - loss: 0.0505 - val_loss: 0.1062
Epoch 8/200
1/1 [==============================] - 0s 48ms/step - loss: 0.0416 - val_loss: 0.0968
Epoch 9/200
1/1 [==============================] - 0s 49ms/step - loss: 0.0346 - val_loss: 0.0879
Epoch 10/200
1/1 [==============================] - 0s 44ms/step -

1/1 [==============================] - 0s 45ms/step - loss: 2.3842e-08 - val_loss: 0.0095
Epoch 161/200
1/1 [==============================] - 0s 42ms/step - loss: 2.3842e-08 - val_loss: 0.0095
Epoch 162/200
1/1 [==============================] - 0s 41ms/step - loss: 1.1921e-08 - val_loss: 0.0095
Epoch 163/200
1/1 [==============================] - 0s 46ms/step - loss: 2.3842e-08 - val_loss: 0.0095
Epoch 164/200
1/1 [==============================] - 0s 45ms/step - loss: 1.7881e-08 - val_loss: 0.0095
Epoch 165/200
1/1 [==============================] - 0s 48ms/step - loss: 0.0000e+00 - val_loss: 0.0095
Epoch 166/200
1/1 [==============================] - 0s 44ms/step - loss: 5.9605e-09 - val_loss: 0.0095
Epoch 167/200
1/1 [==============================] - 0s 43ms/step - loss: 1.1921e-08 - val_loss: 0.0095
Epoch 168/200
1/1 [==============================] - 0s 41ms/step - loss: 1.1921e-08 - val_loss: 0.0095
Epoch 169/200
1/1 [==============================] - 0s 44ms/step - loss: 3.57

...

1/1 [==============================] - 0s 41ms/step - loss: 0.0016 - val_loss: 0.0615
Epoch 39/200
1/1 [==============================] - 0s 44ms/step - loss: 0.0015 - val_loss: 0.0610
Epoch 40/200
1/1 [==============================] - 0s 43ms/step - loss: 0.0014 - val_loss: 0.0604
Epoch 41/200
1/1 [==============================] - 0s 42ms/step - loss: 0.0013 - val_loss: 0.0598
Epoch 42/200
1/1 [==============================] - 0s 41ms/step - loss: 0.0011 - val_loss: 0.0592
Epoch 43/200
1/1 [==============================] - 0s 40ms/step - loss: 0.0010 - val_loss: 0.0586
Epoch 44/200
1/1 [==============================] - 0s 41ms/step - loss: 9.3162e-04 - val_loss: 0.0580
Epoch 45/200
1/1 [==============================] - 0s 41ms/step - loss: 8.3402e-04 - val_loss: 0.0573
Epoch 46/200
1/1 [==============================] - 0s 43ms/step - loss: 7.4462e-04 - val_loss: 0.0566
Epoch 47/200
1/1 [==============================] - 0s 42ms/step - loss: 6.6490e-04 - val_loss: 0.0558
Epoch 4

Epoch 196/200
1/1 [==============================] - 0s 41ms/step - loss: 6.5565e-08 - val_loss: 0.0471
Epoch 197/200
1/1 [==============================] - 0s 43ms/step - loss: 4.7684e-08 - val_loss: 0.0471
Epoch 198/200
1/1 [==============================] - 0s 43ms/step - loss: 3.5763e-08 - val_loss: 0.0471
Epoch 199/200
1/1 [==============================] - 0s 45ms/step - loss: 6.5565e-08 - val_loss: 0.0471
Epoch 200/200
1/1 [==============================] - 0s 43ms/step - loss: 4.1723e-08 - val_loss: 0.0471
1/1 - 0s - loss: 0.0558 - 125ms/epoch - 125ms/step
Epoch 1/200
1/1 [==============================] - 1s 880ms/step - loss: 0.4976 - val_loss: 0.3792
Epoch 2/200
1/1 [==============================] - 0s 43ms/step - loss: 0.3981 - val_loss: 0.3049
Epoch 3/200
1/1 [==============================] - 0s 41ms/step - loss: 0.2936 - val_loss: 0.2274
Epoch 4/200
1/1 [==============================] - 0s 46ms/step - loss: 0.1973 - val_loss: 0.1517
Epoch 5/200
1/1 [===================

1/1 [==============================] - 0s 40ms/step - loss: 1.5682e-05 - val_loss: 0.0145
Epoch 156/200
1/1 [==============================] - 0s 41ms/step - loss: 1.5479e-05 - val_loss: 0.0145
Epoch 157/200
1/1 [==============================] - 0s 45ms/step - loss: 1.5336e-05 - val_loss: 0.0145
Epoch 158/200
1/1 [==============================] - 0s 42ms/step - loss: 1.5157e-05 - val_loss: 0.0145
Epoch 159/200
1/1 [==============================] - 0s 40ms/step - loss: 1.5002e-05 - val_loss: 0.0145
Epoch 160/200
1/1 [==============================] - 0s 46ms/step - loss: 1.4883e-05 - val_loss: 0.0145
Epoch 161/200
1/1 [==============================] - 0s 41ms/step - loss: 1.4734e-05 - val_loss: 0.0145
Epoch 162/200
1/1 [==============================] - 0s 41ms/step - loss: 1.4555e-05 - val_loss: 0.0145
Epoch 163/200
1/1 [==============================] - 0s 46ms/step - loss: 1.4406e-05 - val_loss: 0.0145
Epoch 164/200
1/1 [==============================] - 0s 44ms/step - loss: 1.42

Epoch 116/200
1/1 [==============================] - 0s 39ms/step - loss: 7.5853e-05 - val_loss: 0.0364
Epoch 117/200
1/1 [==============================] - 0s 43ms/step - loss: 7.4136e-05 - val_loss: 0.0363
Epoch 118/200
1/1 [==============================] - 0s 41ms/step - loss: 7.2515e-05 - val_loss: 0.0363
Epoch 119/200
1/1 [==============================] - 0s 39ms/step - loss: 7.0941e-05 - val_loss: 0.0363
Epoch 120/200
1/1 [==============================] - 0s 43ms/step - loss: 6.9463e-05 - val_loss: 0.0363
Epoch 121/200
1/1 [==============================] - 0s 45ms/step - loss: 6.8051e-05 - val_loss: 0.0362
Epoch 122/200
1/1 [==============================] - 0s 42ms/step - loss: 6.6757e-05 - val_loss: 0.0362
Epoch 123/200
1/1 [==============================] - 0s 46ms/step - loss: 6.5523e-05 - val_loss: 0.0362
Epoch 124/200
1/1 [==============================] - 0s 42ms/step - loss: 6.4361e-05 - val_loss: 0.0362
Epoch 125/200
1/1 [==============================] - 0s 41ms/ste

Epoch 195/200
1/1 [==============================] - 0s 46ms/step - loss: 2.3329e-05 - val_loss: 0.0360
Epoch 196/200
1/1 [==============================] - 0s 44ms/step - loss: 2.3013e-05 - val_loss: 0.0359
Epoch 197/200
1/1 [==============================] - 0s 41ms/step - loss: 2.2715e-05 - val_loss: 0.0359
Epoch 198/200
1/1 [==============================] - 0s 43ms/step - loss: 2.2405e-05 - val_loss: 0.0359
Epoch 199/200
1/1 [==============================] - 0s 48ms/step - loss: 2.2089e-05 - val_loss: 0.0359
Epoch 200/200
1/1 [==============================] - 0s 42ms/step - loss: 2.1815e-05 - val_loss: 0.0359
1/1 - 0s - loss: 0.0530 - 128ms/epoch - 128ms/step
Fidelity between ideal and predicted Bloch vectors: 0.9752599103335882


## <font color='gold'>20 samples</font>

In [22]:
inf_predictions_20 = [] # Collecting the predictions in a list

for i in range(10): # for-loop for statistics (average on 10 runs)
    data = generate_Haar_data(1, 100) 
    density_matrix_noise_free = [*map(get_density_matrix, data)] 
    bloch_vectors_noise_free = [*map(Bloch_vector, density_matrix_noise_free)]

    density_matrix_with_noise = []
    for i in range(len(data)):
        single_data_with_noise = flip_error(1, density_matrix_noise_free[i], 'phase', 0.2)
        density_matrix_with_noise.append(single_data_with_noise)
    bloch_vectors_with_noise = [*map(Bloch_vector, density_matrix_with_noise)]
    
   
    # Training, validation and test set
    x_train_list, x_val_list, x_test_list = bloch_vectors_with_noise[:20], bloch_vectors_with_noise[50:90], bloch_vectors_with_noise[90:]
    y_train_list, y_val_list, y_test_list = bloch_vectors_noise_free[:20], bloch_vectors_noise_free[50:90], bloch_vectors_noise_free[90:]


    x_train = tf.convert_to_tensor(x_train_list)
    y_train = tf.convert_to_tensor(y_train_list)


    x_val = tf.convert_to_tensor(x_val_list)
    y_val = tf.convert_to_tensor(y_val_list)

    x_test = tf.convert_to_tensor(x_test_list)
    y_test = tf.convert_to_tensor(y_test_list)
    

    model = tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(3,)),
      tf.keras.layers.Dense(64, activation='relu'), 
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(3),
      tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
       ])
    

    loss_fn = tf.keras.losses.mse
    adam_opt = tf.optimizers.Adam(0.001)
    model.compile(optimizer=adam_opt, 
              loss=infidelity1)

    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=10, epochs=200)
    score = model.evaluate(x_test,  y_test, verbose=2)
    
    y_prediction = model(x_test)
    y_prediction = tf.cast(y_prediction, tf.complex128)
    y_test = tf.cast(y_test, tf.complex128)
    
    fidelities = [] # Collecting the reconstruction fidelities
    for i in range(len(y_prediction)):
        den_mat = density_matrix_from_bloch_vector(y_prediction[i])
        den_mat = np.asarray(den_mat)


        den_mat_id = density_matrix_from_bloch_vector(y_test[i])
        den_mat_id = np.asarray(den_mat_id)
        
        fidelity = fidelity_function(den_mat_id, den_mat)
        fidelities.append(fidelity)
    inf_fid20_c = tf.math.reduce_mean(fidelities)
    inf_fid20 = tf.cast(inf_fid20_c, tf.float64)
    inf_predictions_20.append(inf_fid20)
print(f"Fidelity between ideal and predicted Bloch vectors: {tf.math.reduce_mean(inf_predictions_20).numpy()}")

Epoch 1/200
2/2 [==============================] - 1s 187ms/step - loss: 0.5406 - val_loss: 0.3725
Epoch 2/200
2/2 [==============================] - 0s 48ms/step - loss: 0.4449 - val_loss: 0.3053
Epoch 3/200
2/2 [==============================] - 0s 44ms/step - loss: 0.3513 - val_loss: 0.2523
Epoch 4/200
2/2 [==============================] - 0s 38ms/step - loss: 0.2749 - val_loss: 0.2017
Epoch 5/200
2/2 [==============================] - 0s 40ms/step - loss: 0.2091 - val_loss: 0.1588
Epoch 6/200
2/2 [==============================] - 0s 41ms/step - loss: 0.1597 - val_loss: 0.1335
Epoch 7/200
2/2 [==============================] - 0s 42ms/step - loss: 0.1109 - val_loss: 0.1125
Epoch 8/200
2/2 [==============================] - 0s 44ms/step - loss: 0.0703 - val_loss: 0.1028
Epoch 9/200
2/2 [==============================] - 0s 40ms/step - loss: 0.0479 - val_loss: 0.1011
Epoch 10/200
2/2 [==============================] - 0s 45ms/step - loss: 0.0329 - val_loss: 0.0997
Epoch 11/200
2/2 [

2/2 [==============================] - 0s 41ms/step - loss: 4.0491e-04 - val_loss: 0.0218
Epoch 84/200
2/2 [==============================] - 0s 41ms/step - loss: 3.8689e-04 - val_loss: 0.0217
Epoch 85/200
2/2 [==============================] - 0s 43ms/step - loss: 3.7375e-04 - val_loss: 0.0217
Epoch 86/200
2/2 [==============================] - 0s 39ms/step - loss: 3.5218e-04 - val_loss: 0.0217
Epoch 87/200
2/2 [==============================] - 0s 42ms/step - loss: 3.4126e-04 - val_loss: 0.0216
Epoch 88/200
2/2 [==============================] - 0s 39ms/step - loss: 3.2779e-04 - val_loss: 0.0216
Epoch 89/200
2/2 [==============================] - 0s 39ms/step - loss: 3.1389e-04 - val_loss: 0.0216
Epoch 90/200
2/2 [==============================] - 0s 40ms/step - loss: 3.0174e-04 - val_loss: 0.0216
Epoch 91/200
2/2 [==============================] - 0s 38ms/step - loss: 2.8905e-04 - val_loss: 0.0216
Epoch 92/200
2/2 [==============================] - 0s 40ms/step - loss: 2.7705e-04 - 

2/2 [==============================] - 0s 49ms/step - loss: 3.9044e-05 - val_loss: 0.0189
Epoch 163/200
2/2 [==============================] - 0s 46ms/step - loss: 3.8341e-05 - val_loss: 0.0188
Epoch 164/200
2/2 [==============================] - 0s 48ms/step - loss: 3.7554e-05 - val_loss: 0.0188
Epoch 165/200
2/2 [==============================] - 0s 44ms/step - loss: 3.7208e-05 - val_loss: 0.0188
Epoch 166/200
2/2 [==============================] - 0s 48ms/step - loss: 3.6871e-05 - val_loss: 0.0188
Epoch 167/200
2/2 [==============================] - 0s 44ms/step - loss: 3.5530e-05 - val_loss: 0.0187
Epoch 168/200
2/2 [==============================] - 0s 47ms/step - loss: 3.5116e-05 - val_loss: 0.0187
Epoch 169/200
2/2 [==============================] - 0s 47ms/step - loss: 3.4261e-05 - val_loss: 0.0187
Epoch 170/200
2/2 [==============================] - 0s 44ms/step - loss: 3.3769e-05 - val_loss: 0.0187
Epoch 171/200
2/2 [==============================] - 0s 53ms/step - loss: 3.28

2/2 [==============================] - 0s 36ms/step - loss: 6.9216e-05 - val_loss: 0.0035
Epoch 122/200
2/2 [==============================] - 0s 37ms/step - loss: 6.8638e-05 - val_loss: 0.0035
Epoch 123/200
2/2 [==============================] - 0s 37ms/step - loss: 6.7583e-05 - val_loss: 0.0035
Epoch 124/200
2/2 [==============================] - 0s 40ms/step - loss: 6.5926e-05 - val_loss: 0.0035
Epoch 125/200
2/2 [==============================] - 0s 40ms/step - loss: 6.5202e-05 - val_loss: 0.0035
Epoch 126/200
2/2 [==============================] - 0s 37ms/step - loss: 6.3699e-05 - val_loss: 0.0034
Epoch 127/200
2/2 [==============================] - 0s 38ms/step - loss: 6.3834e-05 - val_loss: 0.0034
Epoch 128/200
2/2 [==============================] - 0s 38ms/step - loss: 6.1351e-05 - val_loss: 0.0034
Epoch 129/200
2/2 [==============================] - 0s 36ms/step - loss: 6.0603e-05 - val_loss: 0.0034
Epoch 130/200
2/2 [==============================] - 0s 40ms/step - loss: 5.93

Epoch 200/200
2/2 [==============================] - 0s 41ms/step - loss: 2.2519e-05 - val_loss: 0.0029
1/1 - 0s - loss: 0.0019 - 139ms/epoch - 139ms/step
Epoch 1/200
2/2 [==============================] - 1s 187ms/step - loss: 0.4498 - val_loss: 0.3666
Epoch 2/200
2/2 [==============================] - 0s 44ms/step - loss: 0.3144 - val_loss: 0.2974
Epoch 3/200
2/2 [==============================] - 0s 39ms/step - loss: 0.2438 - val_loss: 0.2343
Epoch 4/200
2/2 [==============================] - 0s 39ms/step - loss: 0.1728 - val_loss: 0.1877
Epoch 5/200
2/2 [==============================] - 0s 45ms/step - loss: 0.1243 - val_loss: 0.1535
Epoch 6/200
2/2 [==============================] - 0s 39ms/step - loss: 0.0928 - val_loss: 0.1290
Epoch 7/200
2/2 [==============================] - 0s 42ms/step - loss: 0.0692 - val_loss: 0.1082
Epoch 8/200
2/2 [==============================] - 0s 40ms/step - loss: 0.0529 - val_loss: 0.0910
Epoch 9/200
2/2 [==============================] - 0s 40ms/s

...

Epoch 34/200
2/2 [==============================] - 0s 44ms/step - loss: 4.4039e-04 - val_loss: 0.0038
Epoch 35/200
2/2 [==============================] - 0s 43ms/step - loss: 4.0000e-04 - val_loss: 0.0037
Epoch 36/200
2/2 [==============================] - 0s 41ms/step - loss: 3.4917e-04 - val_loss: 0.0037
Epoch 37/200
2/2 [==============================] - 0s 37ms/step - loss: 3.0547e-04 - val_loss: 0.0038
Epoch 38/200
2/2 [==============================] - 0s 40ms/step - loss: 2.6335e-04 - val_loss: 0.0038
Epoch 39/200
2/2 [==============================] - 0s 38ms/step - loss: 2.4468e-04 - val_loss: 0.0038
Epoch 40/200
2/2 [==============================] - 0s 42ms/step - loss: 2.3165e-04 - val_loss: 0.0038
Epoch 41/200
2/2 [==============================] - 0s 44ms/step - loss: 2.1320e-04 - val_loss: 0.0038
Epoch 42/200
2/2 [==============================] - 0s 45ms/step - loss: 1.9580e-04 - val_loss: 0.0037
Epoch 43/200
2/2 [==============================] - 0s 42ms/step - loss: 

Epoch 192/200
2/2 [==============================] - 0s 45ms/step - loss: 1.3828e-06 - val_loss: 0.0030
Epoch 193/200
2/2 [==============================] - 0s 41ms/step - loss: 1.2875e-06 - val_loss: 0.0030
Epoch 194/200
2/2 [==============================] - 0s 41ms/step - loss: 1.2785e-06 - val_loss: 0.0030
Epoch 195/200
2/2 [==============================] - 0s 43ms/step - loss: 1.2338e-06 - val_loss: 0.0030
Epoch 196/200
2/2 [==============================] - 0s 43ms/step - loss: 1.2487e-06 - val_loss: 0.0030
Epoch 197/200
2/2 [==============================] - 0s 48ms/step - loss: 1.2010e-06 - val_loss: 0.0030
Epoch 198/200
2/2 [==============================] - 0s 49ms/step - loss: 1.1951e-06 - val_loss: 0.0030
Epoch 199/200
2/2 [==============================] - 0s 48ms/step - loss: 1.1116e-06 - val_loss: 0.0030
Epoch 200/200
2/2 [==============================] - 0s 44ms/step - loss: 1.1176e-06 - val_loss: 0.0030
1/1 - 0s - loss: 0.0019 - 167ms/epoch - 167ms/step
Epoch 1/200
2

2/2 [==============================] - 0s 39ms/step - loss: 1.9976e-05 - val_loss: 0.0053
Epoch 152/200
2/2 [==============================] - 0s 40ms/step - loss: 1.9121e-05 - val_loss: 0.0053
Epoch 153/200
2/2 [==============================] - 0s 38ms/step - loss: 1.8784e-05 - val_loss: 0.0053
Epoch 154/200
2/2 [==============================] - 0s 38ms/step - loss: 1.8376e-05 - val_loss: 0.0053
Epoch 155/200
2/2 [==============================] - 0s 41ms/step - loss: 1.8051e-05 - val_loss: 0.0053
Epoch 156/200
2/2 [==============================] - 0s 40ms/step - loss: 1.7589e-05 - val_loss: 0.0053
Epoch 157/200
2/2 [==============================] - 0s 40ms/step - loss: 1.7312e-05 - val_loss: 0.0053
Epoch 158/200
2/2 [==============================] - 0s 38ms/step - loss: 1.7062e-05 - val_loss: 0.0053
Epoch 159/200
2/2 [==============================] - 0s 39ms/step - loss: 1.6594e-05 - val_loss: 0.0053
Epoch 160/200
2/2 [==============================] - 0s 39ms/step - loss: 1.66

Epoch 31/200
2/2 [==============================] - 0s 38ms/step - loss: 0.0011 - val_loss: 0.0059
Epoch 32/200
2/2 [==============================] - 0s 38ms/step - loss: 9.6965e-04 - val_loss: 0.0057
Epoch 33/200
2/2 [==============================] - 0s 37ms/step - loss: 9.0986e-04 - val_loss: 0.0055
Epoch 34/200
2/2 [==============================] - 0s 37ms/step - loss: 8.3160e-04 - val_loss: 0.0054
Epoch 35/200
2/2 [==============================] - 0s 38ms/step - loss: 7.3462e-04 - val_loss: 0.0052
Epoch 36/200
2/2 [==============================] - 0s 37ms/step - loss: 6.6257e-04 - val_loss: 0.0051
Epoch 37/200
2/2 [==============================] - 0s 40ms/step - loss: 5.8932e-04 - val_loss: 0.0050
Epoch 38/200
2/2 [==============================] - 0s 38ms/step - loss: 5.1432e-04 - val_loss: 0.0049
Epoch 39/200
2/2 [==============================] - 0s 43ms/step - loss: 4.8320e-04 - val_loss: 0.0049
Epoch 40/200
2/2 [==============================] - 0s 38ms/step - loss: 4.55

Epoch 189/200
2/2 [==============================] - 0s 39ms/step - loss: 3.9905e-06 - val_loss: 0.0030
Epoch 190/200
2/2 [==============================] - 0s 39ms/step - loss: 3.9667e-06 - val_loss: 0.0030
Epoch 191/200
2/2 [==============================] - 0s 40ms/step - loss: 3.7074e-06 - val_loss: 0.0030
Epoch 192/200
2/2 [==============================] - 0s 42ms/step - loss: 3.7342e-06 - val_loss: 0.0030
Epoch 193/200
2/2 [==============================] - 0s 42ms/step - loss: 3.7313e-06 - val_loss: 0.0030
Epoch 194/200
2/2 [==============================] - 0s 37ms/step - loss: 3.6091e-06 - val_loss: 0.0030
Epoch 195/200
2/2 [==============================] - 0s 39ms/step - loss: 3.3855e-06 - val_loss: 0.0030
Epoch 196/200
2/2 [==============================] - 0s 36ms/step - loss: 3.5137e-06 - val_loss: 0.0030
Epoch 197/200
2/2 [==============================] - 0s 43ms/step - loss: 3.3081e-06 - val_loss: 0.0030
Epoch 198/200
2/2 [==============================] - 0s 40ms/ste

## <font color='gold'>30 samples</font>

In [23]:
inf_predictions_30 = [] # Collecting the predictions in a list

for i in range(10): # for-loop for statistics (average on 10 runs)
    data = generate_Haar_data(1, 100) 
    density_matrix_noise_free = [*map(get_density_matrix, data)] 
    bloch_vectors_noise_free = [*map(Bloch_vector, density_matrix_noise_free)]

    density_matrix_with_noise = []
    for i in range(len(data)):
        single_data_with_noise = flip_error(1, density_matrix_noise_free[i], 'phase', 0.2)
        density_matrix_with_noise.append(single_data_with_noise)
    bloch_vectors_with_noise = [*map(Bloch_vector, density_matrix_with_noise)]
    
   
    # Training, validation and test set
    x_train_list, x_val_list, x_test_list = bloch_vectors_with_noise[:30], bloch_vectors_with_noise[50:90], bloch_vectors_with_noise[90:]
    y_train_list, y_val_list, y_test_list = bloch_vectors_noise_free[:30], bloch_vectors_noise_free[50:90], bloch_vectors_noise_free[90:]


    x_train = tf.convert_to_tensor(x_train_list)
    y_train = tf.convert_to_tensor(y_train_list)


    x_val = tf.convert_to_tensor(x_val_list)
    y_val = tf.convert_to_tensor(y_val_list)

    x_test = tf.convert_to_tensor(x_test_list)
    y_test = tf.convert_to_tensor(y_test_list)
    

    model = tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(3,)),
      tf.keras.layers.Dense(64, activation='relu'), 
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(3),
      tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
       ])
    

    loss_fn = tf.keras.losses.mse
    adam_opt = tf.optimizers.Adam(0.001)
    model.compile(optimizer=adam_opt, 
              loss=infidelity1)

    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=6, epochs=200)
    score = model.evaluate(x_test,  y_test, verbose=2)
    
    y_prediction = model(x_test)
    y_prediction = tf.cast(y_prediction, tf.complex128)
    y_test = tf.cast(y_test, tf.complex128)
    
    fidelities = [] # Collecting the reconstruction fidelities
    for i in range(len(y_prediction)):
        den_mat = density_matrix_from_bloch_vector(y_prediction[i])
        den_mat = np.asarray(den_mat)


        den_mat_id = density_matrix_from_bloch_vector(y_test[i])
        den_mat_id = np.asarray(den_mat_id)
        
        fidelity = fidelity_function(den_mat_id, den_mat)
        fidelities.append(fidelity)
    inf_fid30_c = tf.math.reduce_mean(fidelities)
    inf_fid30 = tf.cast(inf_fid30_c, tf.float64)
    inf_predictions_30.append(inf_fid30)
print(f"Fidelity between ideal and predicted Bloch vectors: {tf.math.reduce_mean(inf_predictions_30).numpy()}")

Epoch 1/200
5/5 [==============================] - 1s 52ms/step - loss: 0.4980 - val_loss: 0.3872
Epoch 2/200
5/5 [==============================] - 0s 14ms/step - loss: 0.2829 - val_loss: 0.2668
Epoch 3/200
5/5 [==============================] - 0s 14ms/step - loss: 0.1858 - val_loss: 0.1694
Epoch 4/200
5/5 [==============================] - 0s 14ms/step - loss: 0.0997 - val_loss: 0.0818
Epoch 5/200
5/5 [==============================] - 0s 15ms/step - loss: 0.0481 - val_loss: 0.0452
Epoch 6/200
5/5 [==============================] - 0s 14ms/step - loss: 0.0326 - val_loss: 0.0291
Epoch 7/200
5/5 [==============================] - 0s 13ms/step - loss: 0.0202 - val_loss: 0.0158
Epoch 8/200
5/5 [==============================] - 0s 16ms/step - loss: 0.0099 - val_loss: 0.0082
Epoch 9/200
5/5 [==============================] - 0s 14ms/step - loss: 0.0050 - val_loss: 0.0054
Epoch 10/200
5/5 [==============================] - 0s 16ms/step - loss: 0.0040 - val_loss: 0.0045
Epoch 11/200
5/5 [=

5/5 [==============================] - 0s 15ms/step - loss: 6.4234e-06 - val_loss: 0.0012
Epoch 161/200
5/5 [==============================] - 0s 14ms/step - loss: 6.2207e-06 - val_loss: 0.0012
Epoch 162/200
5/5 [==============================] - 0s 13ms/step - loss: 5.9406e-06 - val_loss: 0.0012
Epoch 163/200
5/5 [==============================] - 0s 14ms/step - loss: 5.6903e-06 - val_loss: 0.0012
Epoch 164/200
5/5 [==============================] - 0s 14ms/step - loss: 5.6446e-06 - val_loss: 0.0012
Epoch 165/200
5/5 [==============================] - 0s 16ms/step - loss: 5.5691e-06 - val_loss: 0.0012
Epoch 166/200
5/5 [==============================] - 0s 16ms/step - loss: 5.9704e-06 - val_loss: 0.0012
Epoch 167/200
5/5 [==============================] - 0s 16ms/step - loss: 5.5532e-06 - val_loss: 0.0012
Epoch 168/200
5/5 [==============================] - 0s 16ms/step - loss: 5.7975e-06 - val_loss: 0.0012
Epoch 169/200
5/5 [==============================] - 0s 17ms/step - loss: 5.42

Epoch 40/200
5/5 [==============================] - 0s 15ms/step - loss: 3.2495e-04 - val_loss: 0.0114
Epoch 41/200
5/5 [==============================] - 0s 13ms/step - loss: 3.0920e-04 - val_loss: 0.0111
Epoch 42/200
5/5 [==============================] - 0s 15ms/step - loss: 2.9692e-04 - val_loss: 0.0112
Epoch 43/200
5/5 [==============================] - 0s 14ms/step - loss: 2.7282e-04 - val_loss: 0.0109
Epoch 44/200
5/5 [==============================] - 0s 14ms/step - loss: 2.5467e-04 - val_loss: 0.0108
Epoch 45/200
5/5 [==============================] - 0s 15ms/step - loss: 2.4779e-04 - val_loss: 0.0107
Epoch 46/200
5/5 [==============================] - 0s 14ms/step - loss: 2.4623e-04 - val_loss: 0.0103
Epoch 47/200
5/5 [==============================] - 0s 14ms/step - loss: 2.2410e-04 - val_loss: 0.0102
Epoch 48/200
5/5 [==============================] - 0s 14ms/step - loss: 2.2169e-04 - val_loss: 0.0103
Epoch 49/200
5/5 [==============================] - 0s 14ms/step - loss: 

Epoch 198/200
5/5 [==============================] - 0s 14ms/step - loss: 8.0804e-06 - val_loss: 0.0070
Epoch 199/200
5/5 [==============================] - 0s 14ms/step - loss: 8.0427e-06 - val_loss: 0.0070
Epoch 200/200
5/5 [==============================] - 0s 15ms/step - loss: 7.5042e-06 - val_loss: 0.0070
1/1 - 0s - loss: 0.0029 - 27ms/epoch - 27ms/step
Epoch 1/200
5/5 [==============================] - 1s 68ms/step - loss: 0.4622 - val_loss: 0.3175
Epoch 2/200
5/5 [==============================] - 0s 16ms/step - loss: 0.2816 - val_loss: 0.1563
Epoch 3/200
5/5 [==============================] - 0s 16ms/step - loss: 0.1388 - val_loss: 0.0946
Epoch 4/200
5/5 [==============================] - 0s 15ms/step - loss: 0.0735 - val_loss: 0.0649
Epoch 5/200
5/5 [==============================] - 0s 16ms/step - loss: 0.0390 - val_loss: 0.0395
Epoch 6/200
5/5 [==============================] - 0s 15ms/step - loss: 0.0215 - val_loss: 0.0245
Epoch 7/200
5/5 [==============================] - 

5/5 [==============================] - 0s 17ms/step - loss: 4.2180e-06 - val_loss: 0.0014
Epoch 157/200
5/5 [==============================] - 0s 16ms/step - loss: 4.1664e-06 - val_loss: 0.0014
Epoch 158/200
5/5 [==============================] - 0s 16ms/step - loss: 4.3551e-06 - val_loss: 0.0014
Epoch 159/200
5/5 [==============================] - 0s 18ms/step - loss: 4.1525e-06 - val_loss: 0.0014
Epoch 160/200
5/5 [==============================] - 0s 16ms/step - loss: 3.9558e-06 - val_loss: 0.0014
Epoch 161/200
5/5 [==============================] - 0s 14ms/step - loss: 4.0929e-06 - val_loss: 0.0014
Epoch 162/200
5/5 [==============================] - 0s 17ms/step - loss: 3.7154e-06 - val_loss: 0.0014
Epoch 163/200
5/5 [==============================] - 0s 17ms/step - loss: 3.5226e-06 - val_loss: 0.0014
Epoch 164/200
5/5 [==============================] - 0s 15ms/step - loss: 3.3836e-06 - val_loss: 0.0014
Epoch 165/200
5/5 [==============================] - 0s 15ms/step - loss: 3.46

...

5/5 [==============================] - 0s 14ms/step - loss: 9.3977e-06 - val_loss: 0.0019
Epoch 108/200
5/5 [==============================] - 0s 13ms/step - loss: 9.3937e-06 - val_loss: 0.0019
Epoch 109/200
5/5 [==============================] - 0s 14ms/step - loss: 8.3049e-06 - val_loss: 0.0019
Epoch 110/200
5/5 [==============================] - 0s 14ms/step - loss: 8.4281e-06 - val_loss: 0.0019
Epoch 111/200
5/5 [==============================] - 0s 13ms/step - loss: 7.8777e-06 - val_loss: 0.0019
Epoch 112/200
5/5 [==============================] - 0s 13ms/step - loss: 7.8817e-06 - val_loss: 0.0019
Epoch 113/200
5/5 [==============================] - 0s 14ms/step - loss: 7.1943e-06 - val_loss: 0.0019
Epoch 114/200
5/5 [==============================] - 0s 14ms/step - loss: 8.3248e-06 - val_loss: 0.0019
Epoch 115/200
5/5 [==============================] - 0s 13ms/step - loss: 7.4089e-06 - val_loss: 0.0019
Epoch 116/200
5/5 [==============================] - 0s 14ms/step - loss: 7.51

Epoch 186/200
5/5 [==============================] - 0s 14ms/step - loss: 7.6294e-07 - val_loss: 0.0018
Epoch 187/200
5/5 [==============================] - 0s 14ms/step - loss: 9.0202e-07 - val_loss: 0.0018
Epoch 188/200
5/5 [==============================] - 0s 14ms/step - loss: 7.5499e-07 - val_loss: 0.0018
Epoch 189/200
5/5 [==============================] - 0s 15ms/step - loss: 8.3248e-07 - val_loss: 0.0018
Epoch 190/200
5/5 [==============================] - 0s 14ms/step - loss: 7.2320e-07 - val_loss: 0.0018
Epoch 191/200
5/5 [==============================] - 0s 14ms/step - loss: 7.8877e-07 - val_loss: 0.0018
Epoch 192/200
5/5 [==============================] - 0s 13ms/step - loss: 6.5962e-07 - val_loss: 0.0018
Epoch 193/200
5/5 [==============================] - 0s 14ms/step - loss: 7.0135e-07 - val_loss: 0.0018
Epoch 194/200
5/5 [==============================] - 0s 13ms/step - loss: 7.6890e-07 - val_loss: 0.0018
Epoch 195/200
5/5 [==============================] - 0s 13ms/ste

5/5 [==============================] - 0s 14ms/step - loss: 1.1663e-06 - val_loss: 0.0034
Epoch 145/200
5/5 [==============================] - 0s 13ms/step - loss: 1.0371e-06 - val_loss: 0.0034
Epoch 146/200
5/5 [==============================] - 0s 14ms/step - loss: 1.2596e-06 - val_loss: 0.0034
Epoch 147/200
5/5 [==============================] - 0s 14ms/step - loss: 1.0908e-06 - val_loss: 0.0034
Epoch 148/200
5/5 [==============================] - 0s 14ms/step - loss: 1.1722e-06 - val_loss: 0.0034
Epoch 149/200
5/5 [==============================] - 0s 15ms/step - loss: 9.9142e-07 - val_loss: 0.0034
Epoch 150/200
5/5 [==============================] - 0s 13ms/step - loss: 1.0232e-06 - val_loss: 0.0034
Epoch 151/200
5/5 [==============================] - 0s 14ms/step - loss: 1.0133e-06 - val_loss: 0.0034
Epoch 152/200
5/5 [==============================] - 0s 14ms/step - loss: 1.1702e-06 - val_loss: 0.0034
Epoch 153/200
5/5 [==============================] - 0s 14ms/step - loss: 9.59

5/5 [==============================] - 0s 13ms/step - loss: 4.7048e-06 - val_loss: 0.0036
Epoch 103/200
5/5 [==============================] - 0s 13ms/step - loss: 4.6074e-06 - val_loss: 0.0037
Epoch 104/200
5/5 [==============================] - 0s 13ms/step - loss: 4.3392e-06 - val_loss: 0.0037
Epoch 105/200
5/5 [==============================] - 0s 13ms/step - loss: 4.0948e-06 - val_loss: 0.0036
Epoch 106/200
5/5 [==============================] - 0s 14ms/step - loss: 3.9319e-06 - val_loss: 0.0036
Epoch 107/200
5/5 [==============================] - 0s 14ms/step - loss: 3.7948e-06 - val_loss: 0.0037
Epoch 108/200
5/5 [==============================] - 0s 13ms/step - loss: 4.3432e-06 - val_loss: 0.0036
Epoch 109/200
5/5 [==============================] - 0s 13ms/step - loss: 4.3670e-06 - val_loss: 0.0036
Epoch 110/200
5/5 [==============================] - 0s 13ms/step - loss: 4.5399e-06 - val_loss: 0.0037
Epoch 111/200
5/5 [==============================] - 0s 13ms/step - loss: 3.42

Epoch 181/200
5/5 [==============================] - 0s 15ms/step - loss: 3.2524e-06 - val_loss: 0.0036
Epoch 182/200
5/5 [==============================] - 0s 13ms/step - loss: 3.3339e-06 - val_loss: 0.0037
Epoch 183/200
5/5 [==============================] - 0s 14ms/step - loss: 2.8749e-06 - val_loss: 0.0037
Epoch 184/200
5/5 [==============================] - 0s 13ms/step - loss: 3.0239e-06 - val_loss: 0.0037
Epoch 185/200
5/5 [==============================] - 0s 14ms/step - loss: 3.1948e-06 - val_loss: 0.0037
Epoch 186/200
5/5 [==============================] - 0s 14ms/step - loss: 2.8511e-06 - val_loss: 0.0037
Epoch 187/200
5/5 [==============================] - 0s 13ms/step - loss: 2.6425e-06 - val_loss: 0.0037
Epoch 188/200
5/5 [==============================] - 0s 13ms/step - loss: 2.5233e-06 - val_loss: 0.0037
Epoch 189/200
5/5 [==============================] - 0s 14ms/step - loss: 3.0617e-06 - val_loss: 0.0037
Epoch 190/200
5/5 [==============================] - 0s 13ms/ste

## <font color='gold'>40 samples</font>

In [24]:
inf_predictions_40 = [] # Collecting the predictions in a list

for i in range(10): # for-loop for statistics (average on 10 runs)
    data = generate_Haar_data(1, 100) 
    density_matrix_noise_free = [*map(get_density_matrix, data)] 
    bloch_vectors_noise_free = [*map(Bloch_vector, density_matrix_noise_free)]

    density_matrix_with_noise = []
    for i in range(len(data)):
        single_data_with_noise = flip_error(1, density_matrix_noise_free[i], 'phase', 0.2)
        density_matrix_with_noise.append(single_data_with_noise)
    bloch_vectors_with_noise = [*map(Bloch_vector, density_matrix_with_noise)]
    
   
    # Training, validation and test set
    x_train_list, x_val_list, x_test_list = bloch_vectors_with_noise[:40], bloch_vectors_with_noise[50:90], bloch_vectors_with_noise[90:]
    y_train_list, y_val_list, y_test_list = bloch_vectors_noise_free[:40], bloch_vectors_noise_free[50:90], bloch_vectors_noise_free[90:]


    x_train = tf.convert_to_tensor(x_train_list)
    y_train = tf.convert_to_tensor(y_train_list)


    x_val = tf.convert_to_tensor(x_val_list)
    y_val = tf.convert_to_tensor(y_val_list)

    x_test = tf.convert_to_tensor(x_test_list)
    y_test = tf.convert_to_tensor(y_test_list)
    

    model = tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(3,)),
      tf.keras.layers.Dense(64, activation='relu'), 
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(3),
      tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
       ])
    

    loss_fn = tf.keras.losses.mse
    adam_opt = tf.optimizers.Adam(0.001)
    model.compile(optimizer=adam_opt, 
              loss=infidelity1)

    history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=8, epochs=200)
    score = model.evaluate(x_test,  y_test, verbose=2)
    
    y_prediction = model(x_test)
    y_prediction = tf.cast(y_prediction, tf.complex128)
    y_test = tf.cast(y_test, tf.complex128)
    
    fidelities = [] # Collecting the reconstruction fidelities
    for i in range(len(y_prediction)):
        den_mat = density_matrix_from_bloch_vector(y_prediction[i])
        den_mat = np.asarray(den_mat)


        den_mat_id = density_matrix_from_bloch_vector(y_test[i])
        den_mat_id = np.asarray(den_mat_id)
        
        fidelity = fidelity_function(den_mat_id, den_mat)
        fidelities.append(fidelity)
    inf_fid40_c = tf.math.reduce_mean(fidelities)
    inf_fid40 = tf.cast(inf_fid40_c, tf.float64)
    inf_predictions_40.append(inf_fid40)
print(f"Fidelity between ideal and predicted Bloch vectors: {tf.math.reduce_mean(inf_predictions_40).numpy()}")

Epoch 1/200
5/5 [==============================] - 1s 49ms/step - loss: 0.3854 - val_loss: 0.2445
Epoch 2/200
5/5 [==============================] - 0s 13ms/step - loss: 0.2077 - val_loss: 0.1409
Epoch 3/200
5/5 [==============================] - 0s 13ms/step - loss: 0.1040 - val_loss: 0.0662
Epoch 4/200
5/5 [==============================] - 0s 13ms/step - loss: 0.0541 - val_loss: 0.0346
Epoch 5/200
5/5 [==============================] - 0s 12ms/step - loss: 0.0319 - val_loss: 0.0211
Epoch 6/200
5/5 [==============================] - 0s 14ms/step - loss: 0.0182 - val_loss: 0.0151
Epoch 7/200
5/5 [==============================] - 0s 18ms/step - loss: 0.0129 - val_loss: 0.0109
Epoch 8/200
5/5 [==============================] - 0s 13ms/step - loss: 0.0087 - val_loss: 0.0080
Epoch 9/200
5/5 [==============================] - 0s 13ms/step - loss: 0.0064 - val_loss: 0.0057
Epoch 10/200
5/5 [==============================] - 0s 13ms/step - loss: 0.0045 - val_loss: 0.0045
Epoch 11/200
5/5 [=

5/5 [==============================] - 0s 13ms/step - loss: 1.6235e-05 - val_loss: 0.0012
Epoch 161/200
5/5 [==============================] - 0s 12ms/step - loss: 1.8112e-05 - val_loss: 0.0012
Epoch 162/200
5/5 [==============================] - 0s 13ms/step - loss: 1.7528e-05 - val_loss: 0.0011
Epoch 163/200
5/5 [==============================] - 0s 13ms/step - loss: 1.6659e-05 - val_loss: 0.0011
Epoch 164/200
5/5 [==============================] - 0s 13ms/step - loss: 1.5976e-05 - val_loss: 0.0012
Epoch 165/200
5/5 [==============================] - 0s 12ms/step - loss: 1.5630e-05 - val_loss: 0.0011
Epoch 166/200
5/5 [==============================] - 0s 12ms/step - loss: 1.5695e-05 - val_loss: 0.0011
Epoch 167/200
5/5 [==============================] - 0s 12ms/step - loss: 1.7662e-05 - val_loss: 0.0011
Epoch 168/200
5/5 [==============================] - 0s 12ms/step - loss: 1.3837e-05 - val_loss: 0.0011
Epoch 169/200
5/5 [==============================] - 0s 13ms/step - loss: 2.03

5/5 [==============================] - 0s 13ms/step - loss: 2.0133e-05 - val_loss: 6.0963e-04
Epoch 116/200
5/5 [==============================] - 0s 13ms/step - loss: 1.9623e-05 - val_loss: 5.9769e-04
Epoch 117/200
5/5 [==============================] - 0s 12ms/step - loss: 1.8276e-05 - val_loss: 5.9762e-04
Epoch 118/200
5/5 [==============================] - 0s 12ms/step - loss: 1.7241e-05 - val_loss: 6.0330e-04
Epoch 119/200
5/5 [==============================] - 0s 13ms/step - loss: 1.7115e-05 - val_loss: 6.0063e-04
Epoch 120/200
5/5 [==============================] - 0s 13ms/step - loss: 1.6508e-05 - val_loss: 5.9955e-04
Epoch 121/200
5/5 [==============================] - 0s 12ms/step - loss: 1.6941e-05 - val_loss: 5.9865e-04
Epoch 122/200
5/5 [==============================] - 0s 13ms/step - loss: 1.7899e-05 - val_loss: 5.8627e-04
Epoch 123/200
5/5 [==============================] - 0s 13ms/step - loss: 1.5764e-05 - val_loss: 5.9258e-04
Epoch 124/200
5/5 [=======================

Epoch 191/200
5/5 [==============================] - 0s 12ms/step - loss: 3.3364e-06 - val_loss: 5.7463e-04
Epoch 192/200
5/5 [==============================] - 0s 12ms/step - loss: 3.2678e-06 - val_loss: 5.7402e-04
Epoch 193/200
5/5 [==============================] - 0s 13ms/step - loss: 3.4258e-06 - val_loss: 5.7596e-04
Epoch 194/200
5/5 [==============================] - 0s 13ms/step - loss: 3.3289e-06 - val_loss: 5.7334e-04
Epoch 195/200
5/5 [==============================] - 0s 12ms/step - loss: 3.3200e-06 - val_loss: 5.6850e-04
Epoch 196/200
5/5 [==============================] - 0s 12ms/step - loss: 3.0652e-06 - val_loss: 5.7750e-04
Epoch 197/200
5/5 [==============================] - 0s 12ms/step - loss: 3.5927e-06 - val_loss: 5.7539e-04
Epoch 198/200
5/5 [==============================] - 0s 13ms/step - loss: 4.1470e-06 - val_loss: 5.6847e-04
Epoch 199/200
5/5 [==============================] - 0s 12ms/step - loss: 3.4943e-06 - val_loss: 5.7839e-04
Epoch 200/200
5/5 [=========

5/5 [==============================] - 0s 13ms/step - loss: 6.7860e-06 - val_loss: 0.0020
Epoch 150/200
5/5 [==============================] - 0s 13ms/step - loss: 7.3239e-06 - val_loss: 0.0020
Epoch 151/200
5/5 [==============================] - 0s 12ms/step - loss: 6.8173e-06 - val_loss: 0.0020
Epoch 152/200
5/5 [==============================] - 0s 12ms/step - loss: 6.8963e-06 - val_loss: 0.0020
Epoch 153/200
5/5 [==============================] - 0s 13ms/step - loss: 7.5310e-06 - val_loss: 0.0020
Epoch 154/200
5/5 [==============================] - 0s 12ms/step - loss: 6.7368e-06 - val_loss: 0.0020
Epoch 155/200
5/5 [==============================] - 0s 12ms/step - loss: 6.8054e-06 - val_loss: 0.0020
Epoch 156/200
5/5 [==============================] - 0s 13ms/step - loss: 6.4164e-06 - val_loss: 0.0020
Epoch 157/200
5/5 [==============================] - 0s 12ms/step - loss: 6.6102e-06 - val_loss: 0.0020
Epoch 158/200
5/5 [==============================] - 0s 13ms/step - loss: 6.96

...

5/5 [==============================] - 0s 14ms/step - loss: 0.0012 - val_loss: 0.0023
Epoch 20/200
5/5 [==============================] - 0s 13ms/step - loss: 0.0011 - val_loss: 0.0021
Epoch 21/200
5/5 [==============================] - 0s 14ms/step - loss: 9.7326e-04 - val_loss: 0.0021
Epoch 22/200
5/5 [==============================] - 0s 12ms/step - loss: 8.4084e-04 - val_loss: 0.0020
Epoch 23/200
5/5 [==============================] - 0s 13ms/step - loss: 7.5149e-04 - val_loss: 0.0020
Epoch 24/200
5/5 [==============================] - 0s 13ms/step - loss: 6.7648e-04 - val_loss: 0.0019
Epoch 25/200
5/5 [==============================] - 0s 12ms/step - loss: 6.1198e-04 - val_loss: 0.0018
Epoch 26/200
5/5 [==============================] - 0s 12ms/step - loss: 5.5023e-04 - val_loss: 0.0018
Epoch 27/200
5/5 [==============================] - 0s 13ms/step - loss: 5.0841e-04 - val_loss: 0.0018
Epoch 28/200
5/5 [==============================] - 0s 12ms/step - loss: 4.6933e-04 - val_loss

Epoch 177/200
5/5 [==============================] - 0s 13ms/step - loss: 1.1401e-05 - val_loss: 0.0012
Epoch 178/200
5/5 [==============================] - 0s 12ms/step - loss: 1.0207e-05 - val_loss: 0.0012
Epoch 179/200
5/5 [==============================] - 0s 13ms/step - loss: 1.0976e-05 - val_loss: 0.0012
Epoch 180/200
5/5 [==============================] - 0s 14ms/step - loss: 1.0085e-05 - val_loss: 0.0012
Epoch 181/200
5/5 [==============================] - 0s 13ms/step - loss: 1.0292e-05 - val_loss: 0.0012
Epoch 182/200
5/5 [==============================] - 0s 13ms/step - loss: 1.0380e-05 - val_loss: 0.0012
Epoch 183/200
5/5 [==============================] - 0s 12ms/step - loss: 9.6843e-06 - val_loss: 0.0011
Epoch 184/200
5/5 [==============================] - 0s 12ms/step - loss: 1.0228e-05 - val_loss: 0.0012
Epoch 185/200
5/5 [==============================] - 0s 13ms/step - loss: 1.0063e-05 - val_loss: 0.0012
Epoch 186/200
5/5 [==============================] - 0s 13ms/ste

5/5 [==============================] - 0s 12ms/step - loss: 9.8571e-06 - val_loss: 7.0091e-04
Epoch 133/200
5/5 [==============================] - 0s 12ms/step - loss: 9.8392e-06 - val_loss: 6.9704e-04
Epoch 134/200
5/5 [==============================] - 0s 13ms/step - loss: 9.6187e-06 - val_loss: 6.9793e-04
Epoch 135/200
5/5 [==============================] - 0s 11ms/step - loss: 9.7409e-06 - val_loss: 6.9980e-04
Epoch 136/200
5/5 [==============================] - 0s 12ms/step - loss: 9.3475e-06 - val_loss: 6.9414e-04
Epoch 137/200
5/5 [==============================] - 0s 12ms/step - loss: 9.2685e-06 - val_loss: 6.9260e-04
Epoch 138/200
5/5 [==============================] - 0s 13ms/step - loss: 9.2119e-06 - val_loss: 6.8834e-04
Epoch 139/200
5/5 [==============================] - 0s 13ms/step - loss: 8.8930e-06 - val_loss: 6.9318e-04
Epoch 140/200
5/5 [==============================] - 0s 12ms/step - loss: 8.5697e-06 - val_loss: 6.9355e-04
Epoch 141/200
5/5 [=======================

5/5 [==============================] - 0s 13ms/step - loss: 2.9354e-05 - val_loss: 5.1776e-04
Epoch 87/200
5/5 [==============================] - 0s 13ms/step - loss: 2.7817e-05 - val_loss: 5.2423e-04
Epoch 88/200
5/5 [==============================] - 0s 12ms/step - loss: 2.7493e-05 - val_loss: 5.2306e-04
Epoch 89/200
5/5 [==============================] - 0s 14ms/step - loss: 2.6056e-05 - val_loss: 5.2308e-04
Epoch 90/200
5/5 [==============================] - 0s 12ms/step - loss: 2.4901e-05 - val_loss: 5.1641e-04
Epoch 91/200
5/5 [==============================] - 0s 13ms/step - loss: 2.3955e-05 - val_loss: 5.1616e-04
Epoch 92/200
5/5 [==============================] - 0s 12ms/step - loss: 2.3556e-05 - val_loss: 5.1586e-04
Epoch 93/200
5/5 [==============================] - 0s 13ms/step - loss: 2.2282e-05 - val_loss: 5.1024e-04
Epoch 94/200
5/5 [==============================] - 0s 13ms/step - loss: 2.1598e-05 - val_loss: 5.1180e-04
Epoch 95/200
5/5 [==============================] 

5/5 [==============================] - 0s 12ms/step - loss: 4.6462e-06 - val_loss: 4.9507e-04
Epoch 163/200
5/5 [==============================] - 0s 12ms/step - loss: 4.3824e-06 - val_loss: 4.9403e-04
Epoch 164/200
5/5 [==============================] - 0s 13ms/step - loss: 4.4003e-06 - val_loss: 4.9051e-04
Epoch 165/200
5/5 [==============================] - 0s 14ms/step - loss: 4.3228e-06 - val_loss: 4.9331e-04
Epoch 166/200
5/5 [==============================] - 0s 12ms/step - loss: 4.1783e-06 - val_loss: 4.9226e-04
Epoch 167/200
5/5 [==============================] - 0s 13ms/step - loss: 4.2468e-06 - val_loss: 4.9087e-04
Epoch 168/200
5/5 [==============================] - 0s 13ms/step - loss: 4.2289e-06 - val_loss: 4.9159e-04
Epoch 169/200
5/5 [==============================] - 0s 12ms/step - loss: 4.1351e-06 - val_loss: 4.8995e-04
Epoch 170/200
5/5 [==============================] - 0s 13ms/step - loss: 4.1291e-06 - val_loss: 4.9190e-04
Epoch 171/200
5/5 [=======================

# <font color='orange'>Plotting the Results</font>

In [25]:
predictions_5_mean = np.mean(predictions_5)
predictions_10_mean = np.mean(predictions_10)
predictions_20_mean = np.mean(predictions_20)
predictions_30_mean = np.mean(predictions_30)
predictions_40_mean = np.mean(predictions_40)

inf_predictions_5_mean = np.mean(inf_predictions_5)
inf_predictions_10_mean = np.mean(inf_predictions_10)
inf_predictions_20_mean = np.mean(inf_predictions_20)
inf_predictions_30_mean = np.mean(inf_predictions_30)
inf_predictions_40_mean = np.mean(inf_predictions_40)

predictions_5_std = np.std(predictions_5)
predictions_10_std = np.std(predictions_10)
predictions_20_std = np.std(predictions_20)
predictions_30_std = np.std(predictions_30)
predictions_40_std = np.std(predictions_40)

inf_predictions_5_std = np.std(inf_predictions_5)
inf_predictions_10_std = np.std(inf_predictions_10)
inf_predictions_20_std = np.std(inf_predictions_20)
inf_predictions_30_std = np.std(inf_predictions_30)
inf_predictions_40_std = np.std(inf_predictions_40)

In [26]:
x = np.array([5, 10, 20, 30, 40])
y = np.array([predictions_5_mean, predictions_10_mean, predictions_20_mean, predictions_30_mean, predictions_40_mean])
z = np.array([inf_predictions_5_mean, inf_predictions_10_mean, inf_predictions_20_mean, inf_predictions_30_mean, inf_predictions_40_mean])